In [1]:
from dask.distributed import Client
client = Client(n_workers=8, threads_per_worker=1, memory_limit='30GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 223.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38385,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 223.52 GiB
Comm: tcp://127.0.0.1:42221,Total threads: 1
Dashboard: http://127.0.0.1:44773/status,Memory: 27.94 GiB
Nanny: tcp://127.0.0.1:40467,


In [1]:
import xarray as xr
import numpy as np
import pickle
import networkx as nx
import torch
import dask
import warnings
warnings.filterwarnings('ignore')
# [distributed, multiprocessing, processes, single-threaded, sync, synchronous, threading, threads]
dask.config.set(scheduler='synchronous')
# chunks = {'time': 10000}
# temperature_2m = xr.open_mfdataset('./data/all_5.625deg/2m_temperature/2m_temperature_*.nc', chunks=chunks)
# u_wind_10m = xr.open_mfdataset('./data/all_5.625deg/10m_u_component_of_wind/10m_u_component_of_wind_*.nc', chunks=chunks)
# v_wind_10m = xr.open_mfdataset('./data/all_5.625deg/10m_v_component_of_wind/10m_v_component_of_wind_*.nc', chunks=chunks)
# geopotential = xr.open_mfdataset('./data/all_5.625deg/geopotential/geopotential_*.nc')
# temperature = xr.open_mfdataset('./data/all_5.625deg/temperature/temperature_*.nc')
# specific_humidity = xr.open_mfdataset('./data/all_5.625deg/specific_humidity/specific_humidity_*.nc')
# u_wind = xr.open_mfdataset('./data/all_5.625deg/u_component_of_wind/u_component_of_wind_*.nc')
# v_wind = xr.open_mfdataset('./data/all_5.625deg/v_component_of_wind/v_component_of_wind_*.nc')

# # geopotential_500 = xr.open_mfdataset('./data/all_5.625deg/geopotential_500/geopotential_*.nc', chunks=chunks) #single level, vs 13 levels
# # vorticity_potential = xr.open_mfdataset('./data/all_5.625deg/potential_vorticity/potential_vorticity_*.nc')
# # vorticity = xr.open_mfdataset('./data/all_5.625deg/vorticity/vorticity_*.nc')
# # relative_humidity = xr.open_mfdataset('./data/all_5.625deg/relative_humidity/relative_humidity_*.nc')


# # temperature_850 = xr.open_mfdataset('./data/all_5.625deg/temperature_850/temperature_850hPa_*.nc') #single level, vs 13 levels
# # solar_radiation = xr.open_mfdataset('./data/all_5.625deg/toa_incident_solar_radiation/toa_incident_solar_radiation_*.nc')
# # cloud_cover = xr.open_mfdataset('./data/all_5.625deg/total_cloud_cover/total_cloud_cover_*.nc')
# # precipitation_total = xr.open_mfdataset('./data/all_5.625deg/total_precipitation/total_precipitation_*.nc')


# data = xr.merge([
#     # temperature_2m,
#     # u_wind_10m,
#     # v_wind_10m,
#     geopotential,
#     temperature,
#     specific_humidity,
#     u_wind,
#     v_wind,
#     # geopotential_500,
#     # vorticity_potential,
#     # vorticity,
#     # relative_humidity,
  
    
#     # temperature_850,
#     # solar_radiation,
#     # cloud_cover,
#     # precipitation_total,

#     ])

data = xr.open_dataset('./data/resampled/3h_backfilled_resampling.nc', engine="h5netcdf")

data

<xarray.Dataset>
Dimensions:  (lon: 64, level: 13, lat: 32, time: 116880)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level    (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T21:00:00
Data variables:
    z        (time, level, lat, lon) float32 ...
    t        (time, level, lat, lon) float32 ...
    q        (time, level, lat, lon) float32 ...
    u        (time, level, lat, lon) float32 ...
    v        (time, level, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-02 22:48:04 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [2]:
data.nbytes // (1024 * 1024 * 1024)

173

In [2]:
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx

sphere_graph = pickle.load(open('./sphere_graph.pickle', 'rb'))
sphere_graph_weighted = pickle.load(open('./sphere_graph_weighted.pickle', 'rb'))
class WeatherDataLoader():
    def __init__(self,
                 data: xr.core.dataarray.DataArray,
                 time: str | slice | list,
                 temporal_resolution: int = 3,
                 min = None,
                 max = None
                 ) -> None:
        """_summary_

        Args:
            data (xarray handle):
                data reference for weatherbench
            lags (int, optional): 
                Input timesteps. Defaults to 112.
            target_length (int, optional): 
                Output timestep length. Defaults to 40.
            temporal_resolution (int, optional): 
                Defines what timesteps to take for both input and output. 1 - every timestep (hour), 3 - every 3rd hour. Defaults to 3.
        """
        super(WeatherDataLoader).__init__()
        self.min, self.max = min, max
        if isinstance(time, list):
            self.data = self._selectTimeRanges(time, data)
        else:
            self.data = data.sel(time=time)
        graph_data = from_networkx(sphere_graph)
        weighted_graph_data = from_networkx(sphere_graph_weighted)
        self.edge_index = graph_data.edge_index
        self.edge_weight = weighted_graph_data.distance
        self.adjacency = nx.adjacency_matrix(sphere_graph_weighted).todense()
        self.adjacency_weighted =nx.adjacency_matrix(sphere_graph_weighted, weight='distance').todense()
        self.timeframe = time
        
        self.stacked = self.stack_data(self.data)
        self.resampled = self.temporal_resampling(self.stacked, temporal_resolution)
        # self.min, self.max = self.getMinMaxValues()
    
    def _selectTimeRanges(self, timeframes, data):
        data_selections = []
        for timeframe in timeframes:
            data_selections.append(data.sel(time=timeframe))
            
        return xr.concat(data_selections,dim='time')
        
    def getMinMaxValues(self):
        if self.min is None or self.max is None:
            self.min, self.max = self.resampled.min(dim="time").compute(num_workers=8), self.resampled.max(dim="time").compute(num_workers=8)
            
        return self.min, self.max
            
        
    def get_data(self):
        # if self.timeframe:
        #     return self.normalize(self.resampled)
    
        # return self.normalize(self.resampled)
        return self.resampled
        
    def temporal_resampling(self, data: xr.core.dataarray.Dataset, temporal_resolution: int) -> xr.core.dataarray.DataArray:
        if temporal_resolution:
            return data.resample(time=f"{temporal_resolution}h").nearest(tolerance="1h")
        return data
        
    def stack_data(self, data: xr.core.dataarray.DataArray) -> xr.core.dataarray.DataArray:
        grouped = data.stack(node=['lat', 'lon'])

        levels = [50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
            1000]
        var_dict = {
            'z': levels,
            # 'pv': levels,
            # 'vo': levels,
            'q': levels,
            't': levels,
            'u': levels,
            'v': levels,
            # 'tisr': None,
            # 'tp': None, 
        }
        generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])

        data_list = []
        def process_levels(d, data_ref):
            _data_list = []    
            for var, levels in d.items():
                try:
                    if levels is None:
                        _data_list.append(data_ref[var].expand_dims({'level': generic_level}, 1))
                    else:    
                        _data_list.append(data_ref[var].sel(level=levels))
                except ValueError:
                    _data_list.append(data_ref[var].expand_dims({'level': generic_level}, 1))
                except KeyError:
                    _data_list.append(data_ref[var])
                    
            return _data_list
        data_list = process_levels(var_dict, grouped) 
        return xr.concat(data_list, 'level').transpose('time', 'node', 'level')
        
    def get_connectivity(self):
        return self.edge_index, self.edge_weight
    
    def normalize(self, data: xr.core.dataarray.DataArray) -> xr.core.dataarray.DataArray:
        # min, max = data.min(dim='time'), data.max(dim='time')
        return (data - self.min) / (self.max - self.min)
        

In [3]:
# resampled = data.resample(time=f"3h").nearest(tolerance="1h")

In [4]:
# resampled.nbytes // (1024**2)

# WARNING: BIG FILEWRITE GOING TO START
### DO NOT RUN WITHOUT GOOD REASON

In [5]:
# resampled.bfill(dim='time').to_netcdf('./data/resampled/3h_backfilled_resampling.nc', engine='h5netcdf')

In [7]:
data.nbytes // (1024**3)

57

In [103]:
# data.sel(time=data.time.isin())
# time=data.isel(time=data.time.isin(['1991', '2004', '2017']))
v1, v2, v3 = data.sel(time='1991'), data.sel(time='2004'), data.sel(time='2017')
# data.isel(time=data.time.isin(['1991', '2004', '2017']))
# xr.merge([v1,v2,v3], compat='identical')
validation = xr.concat([v1,v2,v3], dim='time')

In [104]:
testing = xr.concat([data.sel(time='2012'), data.sel(time='2016')], dim='time')

In [113]:
t1 = data.sel(time=slice('1978', '1990'))
t2 = data.sel(time=slice('1992', '2003'))
t3 = data.sel(time=slice('2005', '2011'))
t4 = data.sel(time=slice('2012', '2015'))
t5 = data.sel(time=slice('2018', '2020'))
# train = []
train = xr.concat([t1,t2,t3,t4,t5], dim='time')

In [114]:
train

<xarray.Dataset>
Dimensions:  (lon: 64, level: 13, lat: 32, time: 315552)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level    (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    z        (time, level, lat, lon) float32 dask.array<chunksize=(8760, 13, 32, 64), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(8760, 13, 32, 64), meta=np.ndarray>
    q        (time, level, lat, lon) float32 dask.array<chunksize=(8760, 13, 32, 64), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(8760, 13, 32, 64), meta=np.ndarray>
    v        (time, level, lat, lon) float32 dask.array<chunksize=(8760, 13, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-02 22:48:04 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [115]:
# global_min, global_max = train.min(dim="time").compute(num_workers=8), train.max(dim="time").compute(num_workers=8)

In [ ]:
# validation (1991, 2004, 2017), testing (2012, 2016, 2020), and training
# (all other years in 1979-2020, inclusive)

In [4]:
min_max = pickle.load(open('./min_max_test.pkl', 'rb'))

In [43]:
train_loader = WeatherDataLoader(
    data=data,
    time=slice('1978', '2020'),
    temporal_resolution=3,
    # min=min_max['min'], 
    # max=min_max['max']
    )

In [44]:
train_loader.get_data().bfill(dim="time").nbytes // (1024 ** 3)

57

In [40]:
train_loader.get_data()

<xarray.DataArray 'z' (time: 116880, node: 2048, level: 65)>
dask.array<getitem, shape=(116880, 2048, 65), dtype=float32, chunksize=(2928, 2048, 13), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T21:00:00
  * node     (node) object MultiIndex
  * lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
  * lon      (node) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [154]:
train_loader = WeatherDataLoader(data=data, time='2000-01-01', temporal_resolution=False, min=min_max['min'], max=min_max['max'])
# train_loader = WeatherDataLoader(
#     data=data,
#     time=[
#         slice('1978', '1990'),
#         slice('1992', '2003'),
#         slice('2005', '2011'),
#         slice('2012', '2015'),
#         slice('2018', '2020')
#         ],
#     temporal_resolution=3,
#     # min=min_max['min'], 
#     # max=min_max['max']
#     )
# test_loader = WeatherDataLoader(data=data, time='2003', temporal_resolution=None)
# validation_loader = WeatherDataLoader(data=data, time='2004', temporal_resolution=3)

In [16]:
def _assert_increasing(x: np.ndarray):
  if not (np.diff(x) > 0).all():
    raise ValueError(f"array is not increasing: {x}")

def _latitude_cell_bounds(x: np.ndarray) -> np.ndarray:
  pi_over_2 = np.array([np.pi / 2], dtype=x.dtype)
  return np.concatenate([-pi_over_2, (x[:-1] + x[1:]) / 2, pi_over_2])


def _cell_area_from_latitude(points: np.ndarray) -> np.ndarray:
  """Calculate the area overlap as a function of latitude."""
  bounds = _latitude_cell_bounds(points)
  _assert_increasing(bounds)
  upper = bounds[1:]
  lower = bounds[:-1]
  # normalized cell area: integral from lower to upper of cos(latitude)
  return np.sin(upper) - np.sin(lower)

In [20]:
data

<xarray.Dataset>
Dimensions:  (lon: 64, level: 13, lat: 32, time: 116880)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level    (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T21:00:00
Data variables:
    z        (time, level, lat, lon) float32 ...
    t        (time, level, lat, lon) float32 ...
    q        (time, level, lat, lon) float32 ...
    u        (time, level, lat, lon) float32 ...
    v        (time, level, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-02 22:48:04 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [19]:
cell_weights = _cell_area_from_latitude(np.deg2rad(data.lat.data))
cell_weights /= cell_weights.mean()
w = data.lat.copy(data=cell_weights)
w

<xarray.DataArray 'lat' (lat: 32)>
array([0.077044, 0.230391, 0.381519, 0.528973, 0.671332, 0.807226, 0.935347,
       1.054459, 1.163416, 1.261169, 1.346776, 1.419413, 1.47838 , 1.52311 ,
       1.553171, 1.568274, 1.568274, 1.553171, 1.52311 , 1.47838 , 1.419413,
       1.346776, 1.261169, 1.163416, 1.054459, 0.935347, 0.807226, 0.671332,
       0.528973, 0.381519, 0.230391, 0.077044])
Coordinates:
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19

In [22]:
data

<xarray.Dataset>
Dimensions:  (lon: 64, level: 13, lat: 32, time: 116880)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level    (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T21:00:00
Data variables:
    z        (time, level, lat, lon) float32 ...
    t        (time, level, lat, lon) float32 ...
    q        (time, level, lat, lon) float32 ...
    u        (time, level, lat, lon) float32 ...
    v        (time, level, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-02 22:48:04 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [31]:
dw =data.sel(time='2000-01-01', level=500).weighted(w)

In [32]:
dw.mean(dim=['lat', 'lon'])

<xarray.Dataset>
Dimensions:  (time: 8)
Coordinates:
    level    int32 500
  * time     (time) datetime64[ns] 2000-01-01 ... 2000-01-01T21:00:00
Data variables:
    z        (time) float64 5.523e+04 5.523e+04 ... 5.522e+04 5.522e+04
    t        (time) float64 257.3 257.3 257.3 257.3 257.3 257.3 257.3 257.3
    q        (time) float64 0.0009679 0.0009724 ... 0.0009719 0.0009774
    u        (time) float64 6.74 6.68 6.703 6.704 6.718 6.742 6.706 6.707
    v        (time) float64 -0.05561 -0.04159 0.01087 ... 0.07145 0.02399

In [188]:
grouped = data.sel(time='2000-01-01', level=500).stack(node=['lat', 'lon'])
grouped2 = data.sel(time='2000-01-02', level=500).stack(node=['lat', 'lon'])
g1 = torch.tensor(grouped.z.data)
g2 = torch.tensor(grouped2.z.data)
# original_shape = torch.tensor(grouped.z.data).reshape((-1, 32, 64))
# original_shape.size()
original_shape = torch.stack((g1, g2)).reshape((2,-1, 32, 64))
weighted_mean(original_shape.refine_names('batch','time', 'lat', 'lon'), weights, dims=['lat', 'lon'])

tensor([[55232.6641, 55227.1758, 55223.0156, 55221.2031, 55225.4062, 55223.5625,
         55222.9297, 55220.7500],
        [55218.0078, 55213.1641, 55210.8125, 55209.8086, 55212.1875, 55211.0391,
         55209.9609, 55207.5430]])

In [70]:
tensor([55232.6680, 55227.1797, 55223.0273, 55221.2109, 55225.4180, 55223.5664,
        55222.9375, 55220.7539])

tensor([0.0770, 0.2304, 0.3815, 0.5290, 0.6713, 0.8072, 0.9353, 1.0545, 1.1634,
        1.2612, 1.3468, 1.4194, 1.4784, 1.5231, 1.5532, 1.5683, 1.5683, 1.5532,
        1.5231, 1.4784, 1.4194, 1.3468, 1.2612, 1.1634, 1.0545, 0.9353, 0.8072,
        0.6713, 0.5290, 0.3815, 0.2304, 0.0770], dtype=torch.float64,
       names=('lat',))

In [91]:
z500 = torch.Tensor(data.sel(time='2000-01-01', level=500).z.data)
named_z500 = z500.refine_names('time', 'lat', 'lon')
weights = torch.tensor(w.data, dtype=named_z500.dtype)
weights = weights.refine_names('lat')

arrays = [named_z500, weights]
all_dims = []
for tens in arrays:
    all_dims += [d for d in tens.names if d not in all_dims]
all_dims

einsum_axes = "abcdefghijklmnopqrstuvwxyz"
dim_map = {d: einsum_axes[i] for i, d in enumerate(all_dims)}
dim_map

{'time': 'a', 'lat': 'b', 'lon': 'c'}

In [109]:
from collections import Counter

def multi_dot(*arrays, dims = None):

    # arrays = [named_z500, weights]
    all_dims = []
    for tens in arrays:
        all_dims += [d for d in tens.names if d not in all_dims]
    all_dims

    einsum_axes = "abcdefghijklmnopqrstuvwxyz"
    dim_map = {d: einsum_axes[i] for i, d in enumerate(all_dims)}
    dim_map
    common_dims = set.intersection(*(set(arr.names) for arr in arrays))
    common_dims

    # dims = ['lat', 'lon']
    if dims is ...:
            dims = all_dims
    elif isinstance(dims, str):
        dims = (dims,)
    elif dims is None:
        # find dimensions that occur more than one times
        dim_counts: Counter = Counter()
        for arr in arrays:
            dim_counts.update(arr.dims)
        dims = tuple(d for d, c in dim_counts.items() if c > 1)

    dot_dims = set(dims)
    broadcast_dims = common_dims - dot_dims
    broadcast_dims

    input_core_dims = [
        [d for d in arr.names if d not in broadcast_dims] for arr in arrays
    ]
    output_core_dims = [
        [d for d in all_dims if d not in dot_dims and d not in broadcast_dims]
    ]
    subscripts_list = [
        "..." + "".join(dim_map[d] for d in ds) for ds in input_core_dims
    ]
    subscripts = ",".join(subscripts_list)
    subscripts += "->..." + "".join(dim_map[d] for d in output_core_dims[0])

    return torch.einsum(subscripts, *[a.rename(None) for a in arrays])

In [133]:
def weighted_sum(data, weights, dims=None):
    return multi_dot(data, weights, dims=dims)

def sum_of_weights(data, weights, dims=None):
    summed_weights = multi_dot((~torch.isnan(data)).int().float(), weights, dims=dims)
    valid_weights = summed_weights != 0.0
    
    return summed_weights.where(valid_weights, 1e-10)

In [136]:
from collections import Counter

def multi_dot(*arrays, dims = None):

    # arrays = [named_z500, weights]
    all_dims = []
    for tens in arrays:
        all_dims += [d for d in tens.names if d not in all_dims]
    all_dims

    einsum_axes = "abcdefghijklmnopqrstuvwxyz"
    dim_map = {d: einsum_axes[i] for i, d in enumerate(all_dims)}
    dim_map
    common_dims = set.intersection(*(set(arr.names) for arr in arrays))
    common_dims

    # dims = ['lat', 'lon']
    if dims is ...:
            dims = all_dims
    elif isinstance(dims, str):
        dims = (dims,)
    elif dims is None:
        # find dimensions that occur more than one times
        dim_counts: Counter = Counter()
        for arr in arrays:
            dim_counts.update(arr.dims)
        dims = tuple(d for d, c in dim_counts.items() if c > 1)

    dot_dims = set(dims)
    broadcast_dims = common_dims - dot_dims
    broadcast_dims

    input_core_dims = [
        [d for d in arr.names if d not in broadcast_dims] for arr in arrays
    ]
    output_core_dims = [
        [d for d in all_dims if d not in dot_dims and d not in broadcast_dims]
    ]
    subscripts_list = [
        "..." + "".join(dim_map[d] for d in ds) for ds in input_core_dims
    ]
    subscripts = ",".join(subscripts_list)
    subscripts += "->..." + "".join(dim_map[d] for d in output_core_dims[0])

    return torch.einsum(subscripts, *[a.rename(None) for a in arrays])

def weighted_sum(data, weights, dims=None):
    return multi_dot(data, weights, dims=dims)

def sum_of_weights(data, weights, dims=None):
    summed_weights = multi_dot((~torch.isnan(data)).int().float(), weights, dims=dims)
    valid_weights = summed_weights != 0.0
    
    return summed_weights.where(valid_weights, 1e-10)

def weighted_mean(data, weights, dims=None):
    weighted_sum_ = weighted_sum(data, weights, dims=dims)
    sum_o_w = sum_of_weights(data, weights, dims=dims)

    return weighted_sum_ / sum_o_w

In [152]:
weighted_mean(named_z500, weights, dims=['lat', 'lon']).size()

torch.Size([8])

In [190]:
from collections import Counter

def multi_dot(*arrays, dims = None):

    # arrays = [named_z500, weights]
    all_dims = []
    for tens in arrays:
        all_dims += [d for d in tens.names if d not in all_dims]
    all_dims

    einsum_axes = "abcdefghijklmnopqrstuvwxyz"
    dim_map = {d: einsum_axes[i] for i, d in enumerate(all_dims)}
    dim_map
    common_dims = set.intersection(*(set(arr.names) for arr in arrays))
    common_dims

    # dims = ['lat', 'lon']
    if dims is ...:
            dims = all_dims
    elif isinstance(dims, str):
        dims = (dims,)
    elif dims is None:
        # find dimensions that occur more than one times
        dim_counts: Counter = Counter()
        for arr in arrays:
            dim_counts.update(arr.dims)
        dims = tuple(d for d, c in dim_counts.items() if c > 1)

    dot_dims = set(dims)
    broadcast_dims = common_dims - dot_dims
    broadcast_dims

    input_core_dims = [
        [d for d in arr.names if d not in broadcast_dims] for arr in arrays
    ]
    output_core_dims = [
        [d for d in all_dims if d not in dot_dims and d not in broadcast_dims]
    ]
    subscripts_list = [
        "..." + "".join(dim_map[d] for d in ds) for ds in input_core_dims
    ]
    subscripts = ",".join(subscripts_list)
    subscripts += "->..." + "".join(dim_map[d] for d in output_core_dims[0])

    return torch.einsum(subscripts, *[a.rename(None) for a in arrays])

def weighted_sum(data, weights, dims=None):
    return multi_dot(data, weights, dims=dims)

def sum_of_weights(data, weights, dims=None):
    summed_weights = multi_dot((~torch.isnan(data)).int().float(), weights, dims=dims)
    valid_weights = summed_weights != 0.0
    
    return summed_weights.where(valid_weights, 1e-10)

def weighted_mean(data, weights, dims=None):
    weighted_sum_ = weighted_sum(data, weights, dims=dims)
    sum_o_w = sum_of_weights(data, weights, dims=dims)

    return weighted_sum_ / sum_o_w

def _spatial_average_l2_norm(diff):
    return torch.sqrt(weighted_mean(diff**2, weights, dims=['lat', 'lon']))

In [14]:
data.sel(time=slice('2000', '2002'), level=500, lat=-87.1875, lon=0.0).z.data

array([49469.062, 49482.152, 49482.152, ..., 50545.168, 50555.008,
       50587.81 ], dtype=float32)

In [155]:
node_data = torch.tensor(train_loader.get_data()[:, :, 7].data, names=('time', 'node'))

In [156]:
node_data.size()

torch.Size([8, 2048])

In [157]:
weighted_mean(node_data, weights, dims=['node']).size()

torch.Size([8, 32])

In [ ]:
data.sel(time='2000', )

In [22]:
min_val, max_val = train_loader.getMinMaxValues()
# pickle.dump({'min': min_val, 'max': max_val}, open('./min_max_train_full.pkl', 'wb'))

In [46]:
# min_val, max_val = train_loader.getMinMaxValues()
validation_loader = WeatherDataLoader(data=data, time='2003', temporal_resolution=3, min=min_val, max=max_val)

In [49]:
validation_loader.get_data().nbytes // (1024**2)

1482

In [56]:
validation_loader.get_data().reset_index('node')

<xarray.DataArray 'z' (time: 2920, node: 2048, level: 65)>
dask.array<getitem, shape=(2920, 2048, 65), dtype=float32, chunksize=(2920, 2048, 13), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 2003-01-01 ... 2003-12-31T21:00:00
    lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
    lon      (node) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Dimensions without coordinates: node
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [77]:
# file writting test
validation_loader.get_data().reset_index('node').to_netcdf('./data/resampled/test_write_h5.nc', engine='h5netcdf')

In [86]:
train_loader.get_data()

<xarray.DataArray 'z' (time: 116880, node: 2048, level: 65)>
dask.array<getitem, shape=(116880, 2048, 65), dtype=float32, chunksize=(2928, 2048, 13), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T21:00:00
  * node     (node) object MultiIndex
  * lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
  * lon      (node) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [78]:
test_ds = xr.open_dataset('./data/resampled/test_write_h5.nc')

In [85]:
test_ds

<xarray.Dataset>
Dimensions:  (level: 65, time: 2920, node: 2048)
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 2003-01-01 ... 2003-12-31T21:00:00
    lat      (node) float64 ...
    lon      (node) float64 ...
Dimensions without coordinates: node
Data variables:
    z        (time, node, level) float32 ...

In [66]:
validation_loader.get_data()

<xarray.DataArray 'z' (time: 2920, node: 2048, level: 65)>
dask.array<getitem, shape=(2920, 2048, 65), dtype=float32, chunksize=(2920, 2048, 13), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 2003-01-01 ... 2003-12-31T21:00:00
  * node     (node) object MultiIndex
  * lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
  * lon      (node) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [68]:
test_ds.close()

In [153]:
transposed = train_loader.get_data()
transposed

<xarray.DataArray 'z' (time: 8768, node: 2048, level: 65)>
array([[[ 1.96816266e+05,  1.50654312e+05,  1.25160023e+05, ...,
         -3.30151486e+00, -3.31633925e+00, -3.31040955e+00],
        [ 1.96741000e+05,  1.50611766e+05,  1.25130570e+05, ...,
         -3.68991899e+00, -3.66916466e+00, -3.68102407e+00],
        [ 1.96662469e+05,  1.50569219e+05,  1.25104391e+05, ...,
         -3.90042806e+00, -3.91525245e+00, -3.91228747e+00],
        ...,
        [ 1.87896047e+05,  1.49018172e+05,  1.25389078e+05, ...,
          8.11635971e+00,  6.80885649e+00,  1.67067945e+00],
        [ 1.87932047e+05,  1.49054156e+05,  1.25428344e+05, ...,
          8.73604202e+00,  7.09346724e+00,  1.50755334e+00],
        [ 1.87968031e+05,  1.49090156e+05,  1.25461070e+05, ...,
          9.31122303e+00,  8.47213936e+00,  4.68588543e+00]],

       [[ 1.96832625e+05,  1.50651031e+05,  1.25182930e+05, ...,
         -3.24221635e+00, -3.26297092e+00, -3.25407600e+00],
        [ 1.96757375e+05,  1.50611766e+05,  1.25153477e+05, ...,
         -3.54167318e+00, -3.51202393e+00, -3.52388382e+00],
        [ 1.96682094e+05,  1.50569219e+05,  1.25124031e+05, ...,
         -3.69288397e+00, -3.71067309e+00, -3.70770812e+00],
...
          2.06917877e+01,  2.61888428e+01,  1.74711552e+01],
        [ 1.89533031e+05,  1.50183469e+05,  1.26295602e+05, ...,
          2.17981529e+01,  2.74081764e+01,  1.89540405e+01],
        [ 1.89533031e+05,  1.50190031e+05,  1.26312000e+05, ...,
          2.26844120e+01,  2.75703888e+01,  1.99909058e+01]],

       [[ 2.01944344e+05,  1.55834828e+05,  1.29090117e+05, ...,
         -1.26362324e-01, -1.46636009e-01, -1.32154703e-01],
        [ 2.01941062e+05,  1.55815141e+05,  1.29067156e+05, ...,
         -5.28939664e-01, -5.08665979e-01, -5.20250976e-01],
        [ 2.01934516e+05,  1.55795469e+05,  1.29044195e+05, ...,
         -7.25883842e-01, -7.34572530e-01, -7.17195153e-01],
        ...,
        [ 1.89457594e+05,  1.50160516e+05,  1.26256242e+05, ...,
          2.25164261e+01,  2.63307819e+01,  1.81315079e+01],
        [ 1.89460875e+05,  1.50173625e+05,  1.26285758e+05, ...,
          2.29740448e+01,  2.69621582e+01,  1.92205009e+01],
        [ 1.89460875e+05,  1.50180188e+05,  1.26308719e+05, ...,
          2.32636700e+01,  2.76340904e+01,  2.00024948e+01]]],
      dtype=float32)
Coordinates:
  * level    (level) int32 50 100 150 200 250 300 ... 500 600 700 850 925 1000
  * time     (time) datetime64[ns] 2000-01-01 ... 2002-12-31T21:00:00
  * node     (node) object MultiIndex
  * lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
  * lon      (node) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [116]:
# pickle.dump({'min': global_min, 'max': global_max}, open('./min_max_global.pkl', 'wb'))

In [7]:
# from tsl.data.preprocessing import Scaler
# from tsl.typing import TensArray
# import os

# bias = None
# scale = None
# def get_bias_and_scale(mins, max):
#     bias = mins
#     scale = max - mins
#     return bias, scale    

# mins = None
# max = None
# def get_norm_scaler_data(loader):
#     dataset = loader.get_data()
#     min_name = f'./constants/min_{loader.timeframe}_{dataset.shape[-1]}.nc'
#     max_name = f'./constants/max_{loader.timeframe}_{dataset.shape[-1]}.nc'
    
#     # if mins is not None and max is not None:
#     #     return get_bias_and_scale(mins, max)
    
#     if os.path.exists(min_name) and os.path.exists(max_name):
#         mins = xr.load_dataset(min_name).load().values
#         max = xr.load_dataset(max_name).load().values
    
#     else:
#         mins = dataset.bfill(dim='time').min(dim='time').load().values
#         max = dataset.bfill(dim='time').max(dim='time').load().values
        
        
#     return get_bias_and_scale(mins, max)
        

# bias, scale = get_norm_scaler_data(train_loader)

# class WeatherNormalizationScaler(Scaler):
#     def fit(self, x: TensArray, *args, **kwargs):
#         return self
# scalers = {
#     'target': WeatherNormalizationScaler(bias, scale)
# }

In [8]:
from torch import Tensor
from torch_sparse import SparseTensor
from tsl.data.data import Data as TslData, StorageView as TslStorageView
from torch_geometric.typing import Adj
from typing import (Any, Callable, Dict, Iterable, Iterator, List, Mapping,
                    Optional, Tuple, Union)

def pattern_size_repr(key: str,
                      x: Union[Tensor, SparseTensor, xr.core.dataarray.DataArray],
                      pattern: str = None):
    if isinstance(x, xr.core.dataarray.DataArray):
        x = torch.Tensor(x.load().data)
    if pattern is not None:
        pattern = pattern.replace(' ', '')
        out = str([
            f'{token}={size}' if not token.isnumeric() else str(size)
            for token, size in zip(pattern, get_size(x))
        ])
    else:
        out = str(list(get_size(x)))
    out = f"{key}={out}".replace("'", '')
    return out

def get_size(x: Union[Tensor, SparseTensor]) -> Tuple:
    if isinstance(x, Tensor):
        return tuple(x.size())
    elif isinstance(x, SparseTensor):
        return tuple(x.sizes())
    
class StorageView(TslStorageView):
    def __repr__(self) -> str:
        cls = self.__class__.__name__
        info = [pattern_size_repr(k, v) for k, v in self.items()]
        return '{}({})'.format(cls, ', '.join(info))
    
class Data(TslData):
    def __init__(self,
                 input: Optional[Mapping] = None,
                 target: Optional[Mapping] = None,
                 edge_index: Optional[Adj] = None,
                 edge_weight: Optional[Tensor] = None,
                 mask: Optional[Tensor] = None,
                 transform: Optional[Mapping] = None,
                 pattern: Optional[Mapping] = None,
                 **kwargs):
        input = input if input is not None else dict()
        target = target if target is not None else dict()
        super(Data, self).__init__(**input,
                                   **target,
                                   edge_index=edge_index,
                                   edge_weight=edge_weight,
                                   **kwargs)
        # Set 'input' as view on input keys
        self.__dict__['input'] = StorageView(self._store, input.keys())
        # Set 'target' as view on target keys
        self.__dict__['target'] = StorageView(self._store, target.keys())
        # Add mask
        self.mask = mask  # noqa
        # Add transform modules
        transform = transform if transform is not None else dict()
        self.transform: dict = transform  # noqa
        # Add patterns
        self.__dict__['pattern'] = dict()
        if pattern is not None:
            self.pattern.update(pattern)

    def __repr__(self) -> str:
        cls = self.__class__.__name__
        inputs = [
            pattern_size_repr(k, v, self.pattern.get(k))
            for k, v in self.input.items()
        ]
        inputs = 'input=({})'.format(', '.join(inputs))
        targets = [
            pattern_size_repr(k, v, self.pattern.get(k))
            for k, v in self.target.items()
        ]
        targets = 'target=({})'.format(', '.join(targets))
        info = [inputs, targets, "has_mask={}".format(self.has_mask)]
        if self.has_transform:
            info += ["transform=[{}]".format(', '.join(self.transform.keys()))]
        return '{}(\n  {}\n)'.format(cls, ',\n  '.join(info))

In [191]:
from numpy import ndarray
import pandas as pd
from copy import deepcopy
from types import ModuleType
from tsl.ops.pattern import broadcast, check_pattern, outer_pattern, _infer_backend, _get_select_fn, _parse_indices
from tsl.utils.casting import parse_index
from tsl.data.synch_mode import HORIZON, STATIC, WINDOW
from tsl.data import StaticBatch
from tsl.data.preprocessing.scalers import ScalerModule, Scaler
from tsl.typing import (DataArray, IndexSlice, SparseTensArray, TemporalIndex,
                        TensArray)
from typing import (Callable, Iterable, List, Literal, Mapping, Optional,
                    Tuple, Union)
from tsl.data import SpatioTemporalDataset as TslSpatioTemporalDataset, BatchMap
from pandas import DatetimeIndex
from tsl.data.batch_map import BatchMap, BatchMapItem
from contextlib import contextmanager
from torch.utils.data import Dataset
from tsl.data.mixin import DataParsingMixin
from tsl.data.synch_mode import HORIZON, STATIC, WINDOW, SynchMode
from tsl.ops.connectivity import reduce_graph

def take(x: Union[np.ndarray, torch.Tensor, xr.core.dataarray.DataArray],
         pattern: str,
         time_index: Union[List, ndarray, Tensor] = None,
         node_index: Union[List, ndarray, Tensor] = None,
         edge_mask: Union[List, ndarray, Tensor] = None,
         *,
         backend: ModuleType = None):
    # print(x)
    # if isinstance(x, xr.core.dataarray.DataArray):
    #     x = torch.Tensor(x.load().data)
    backend = _infer_backend(torch.ones(1), backend)
    dims = check_pattern(pattern, split=True)

    select = _get_select_fn(backend)
    time_index, node_index, edge_index = _parse_indices(backend,
                                                        time_index=time_index,
                                                        node_index=node_index,
                                                        edge_mask=edge_mask)

    # assume that 't' can only be first dimension, then allow multidimensional
    # temporal indexing
    pad_dim = 0
    if dims[0] == 't':
        pad_dim = 1
        if time_index is not None:
            # time_index can be multidimensional
            pad_dim = time_index.ndim
            # pad pattern with 'batch' dimensions
            dims = ['b'] * (pad_dim - 1) + dims
            # print('time index', time_index, x)
            # print('load time indexted x', x[time_index])
            if isinstance(x, xr.core.dataarray.DataArray):
                time_index = xr.DataArray(time_index)
            x = torch.Tensor(x[time_index].bfill(dim='dim_0').load().data)

    # broadcast array/tensor to pattern according to backend
    for pos, dim in list(enumerate(dims))[pad_dim:]:
        if dim == 'n' and node_index is not None:
            x = select(x, node_index, pos)
        elif dim == 'e' and edge_index is not None:
            x = select(x, edge_index, pos)

    return x

_WINDOWING_ = {
    'window': ['window', 'window_lag'],
    'horizon': ['window', 'delay', 'horizon', 'horizon_lag'],
    'indices': ['target', 'window', 'delay', 'horizon', 'stride'],
    'all': [
        'target', 'window', 'delay', 'horizon', 'stride', 'horizon_lag',
        'window_lag'
    ]
}

class SpatioTemporalDataset(Dataset, DataParsingMixin):
    r"""Base class for structures that are bridges between Datasets and Models.

    A :class:`SpatioTemporalDataset` takes as input a
    :class:`~tsl.datasets.prototypes.Dataset` and
    build a proper structure to feed deep models.

    Args:
        target (DataArray): Data relative to the primary channels.
        index (TemporalIndex, optional): Temporal indices for the data.
            (default: :obj:`None`)
        mask (DataArray, optional): Boolean mask denoting if signal in data is
            valid (1) or not (0).
            (default: :obj:`None`)
        connectivity (SparseTensArray, tuple, optional): The adjacency matrix
            defining nodes' relational information. It can be either a
            dense/sparse matrix :math:`\mathbf{A} \in \mathbb{R}^{N \times N}`
            or an (:obj:`edge_index` :math:`\in \mathbb{N}^{2 \times E}`,
            :obj:`edge_weight` :math:`\in \mathbb{R}^{E})` tuple. The input
            layout will be preserved (e.g., a sparse matrix will be stored as a
            :class:`torch_sparse.SparseTensor`). In any case, the connectivity
            will be stored in the attribute :obj:`edge_index`, and the weights
            will be eventually stored as :obj:`edge_weight`.
            (default: :obj:`None`)
        covariates (dict, optional): Dictionary of exogenous channels with
            label. An :obj:`exogenous` element is a temporal array with node- or
            graph-level channels which are covariates to the main signal. The
            temporal dimension must be equal to the temporal dimension of data,
            as well as the number of nodes if the exogenous is node-level.
            (default: :obj:`None`)
        input_map (BatchMap or dict, optional): Defines how data (i.e., the
            target and the covariates) are mapped to dataset sample input. Keys
            in the mapping are keys in both :obj:`item` and :obj:`item.input`,
            while values are :obj:`~tsl.data.new.BatchMapItem`.
            (default: :obj:`None`)
        target_map (BatchMap or dict, optional): Defines how data (i.e., the
            target and the covariates) are mapped to dataset sample target. Keys
            in the mapping are keys in both :obj:`item` and :obj:`item.target`,
            while values are :obj:`~tsl.data.new.BatchMapItem`.
            (default: :obj:`None`)
        auxiliary_map (BatchMap or dict, optional): Defines how data (i.e., the
            target and the covariates) are added as additional attributes to the
            dataset sample. Keys in the mapping are keys only in :obj:`item`,
            while values are :obj:`~tsl.data.new.BatchMapItem`.
            (default: :obj:`None`)
        scalers (Mapping or None): Dictionary of scalers that must be used for
            data preprocessing.
            (default: :obj:`None`)
        trend (DataArray, optional): Trend paired with main signal. Must be of
            the same shape of `data`.
            (default: :obj:`None`)
        transform (callable, optional): A function/transform that takes in a
            :class:`tsl.data.Data` object and returns a transformed version.
            The data object will be transformed before every access.
            (default: :obj:`None`)
        window (int): Length (in number of steps) of the lookback window.
            (default: 12)
        horizon (int): Length (in number of steps) of the prediction horizon.
            (default: 1)
        delay (int): Offset (in number of steps) between end of window and start
            of horizon.
            (default: 0)
        stride (int): Offset (in number of steps) between a sample and the next
            one.
            (default: 1)
        window_lag (int): Sampling frequency (in number of steps) in lookback
            window.
            (default: 1)
        horizon_lag (int): Sampling frequency (in number of steps) in prediction
            horizon.
            (default: 1)
        precision (int or str, optional): The float precision to store the data.
            Can be expressed as number (16, 32, or 64) or string ("half",
            "full", "double").
            (default: 32)
        name (str, optional): The (optional) name of the dataset.
    """

    def __init__(self,
                 target: DataArray,
                 index: Optional[TemporalIndex] = None,
                 mask: Optional[DataArray] = None,
                 connectivity: Optional[Union[SparseTensArray,
                                              Tuple[DataArray]]] = None,
                 covariates: Optional[Mapping[str, DataArray]] = None,
                 input_map: Optional[Union[Mapping, BatchMap]] = None,
                 target_map: Optional[Union[Mapping, BatchMap]] = None,
                 auxiliary_map: Optional[Union[Mapping, BatchMap]] = None,
                 scalers: Optional[Mapping[str, Scaler]] = None,
                 trend: Optional[DataArray] = None,
                 transform: Optional[Callable] = None,
                 window: int = 12,
                 horizon: int = 1,
                 delay: int = 0,
                 stride: int = 1,
                 window_lag: int = 1,
                 horizon_lag: int = 1,
                 precision: Union[int, str] = 32,
                 name: Optional[str] = None):
        super(SpatioTemporalDataset, self).__init__()

        # Set info
        self.name = name if name is not None else self.__class__.__name__
        self.precision = precision

        # Store windowing information
        self.window = window
        self.delay = delay
        self.horizon = horizon
        self.stride = stride
        self.window_lag = window_lag
        self.horizon_lag = horizon_lag

        # Initialize private data holders
        self._covariates = dict()
        self._indices = None

        # Initialize batch maps
        self.input_map = BatchMap()
        self.target_map = BatchMap()
        self.auxiliary_map = BatchMap()

        # Store preprocessing modules
        self.scalers: dict = dict()
        self.trend: Optional[Tensor] = None
        self.transform = transform
        # cache scalers for composed batch items
        self._batch_scalers: dict = dict()
        # handle trend as bias in target scaler, so cache actual one (if any)
        # to restore after trend update/deletion
        self.__target_bias: Optional[Tensor] = None

        # Store time information
        # if index is not explicitly passed and data is a dataframe, defaults to
        # data's index
        if index is None and isinstance(target, pd.DataFrame):
            if isinstance(target.index, DatetimeIndex):
                index = target.index
        self.index = index

        # Set dataset's target signals
        # self.target: Tensor = self._parse_target(target)
        self.target: Tensor = target
        # print(self.target.shape)

        # Store mask
        self.mask: Optional[Tensor] = None
        self.set_mask(mask)

        # Store adj
        self.edge_index: Optional[Adj] = None
        self.edge_weight: Optional[Tensor] = None
        self.set_connectivity(connectivity)

        # Store covariates (e.g., exogenous and attributes)
        self.reset_input_map()
        if covariates is not None:
            for name, value in covariates.items():
                self.add_covariate(name, **self._value_to_kwargs(value))

        # Updated input map (i.e., how to map data, exogenous and attribute
        # inside item.input)
        if input_map is not None:
            self.set_input_map(input_map)

        # Updated target map (i.e., how to map data, exogenous and attribute
        # inside item.target)
        self.reset_target_map()
        if target_map is not None:
            self.set_target_map(target_map)

        # Updated auxiliary map (i.e., how to map data, exogenous and attribute
        # inside item)
        if auxiliary_map is not None:
            self.set_auxiliary_map(auxiliary_map)

        # A scaler is a module that transforms data with a linear operation
        if scalers is not None:
            for k, v in scalers.items():
                self.add_scaler(k, v)

        # Target's trend (i.e., 't n f' tensor to be removed when target
        # is preprocessed)
        if trend is not None:
            self.set_trend(trend)
    def __getitem__(self, item) -> Data:
        if isinstance(item, int) and item < 0:
            # compute item's actual index
            item = self._indices.size(0) + item
        else:
            # convert slice to indexes
            item = parse_index(item, length=self.n_samples, layout='index')
        item = self.get(item)
        if self.transform is not None:
            item = self.transform(item)
        return item
    @property
    def n_steps(self) -> int:
        """Total number of time steps in the dataset."""
        # return self.target.size(0)
        return self.target.shape[0]

    @property
    def n_nodes(self) -> int:
        """Number of nodes in the dataset."""
        return self.target.shape[1]

    @property
    def n_channels(self) -> int:
        """Number of channels in dataset's target."""
        return self.target.shape[-1]
    
    @property
    def shape(self) -> tuple:
        """Shape of the target tensor."""
        # return tuple(self.target.size())
        return self.target.shape
    
    def get(self, item):
        # check if item is scalar or vector
        ndim = item.ndim if isinstance(item, Tensor) else 0
        
        if ndim == 0:  # get a single item
            sample = Data(pattern=self.batch_patterns)
        elif ndim == 1:  # get batch of items
            pattern = {
                name: ('b ' + pattern) if 't' in pattern else pattern
                for name, pattern in self.batch_patterns.items()
            }
            sample = StaticBatch(pattern=pattern, size=item.size(0))
        else:
            raise RuntimeError(f"Too many dimensions for index ({ndim}).")

        # get input synchronized with window
        if self.window > 0:
            wdw_idxs = self.get_window_indices(item)
            self._add_to_sample(sample, WINDOW, 'input', time_index=wdw_idxs)
            self._add_to_sample(sample, WINDOW, 'target', time_index=wdw_idxs)
            self._add_to_sample(sample,
                                WINDOW,
                                'auxiliary',
                                time_index=wdw_idxs)

        # get input synchronized with horizon
        hrz_idxs = self.get_horizon_indices(item)
        self._add_to_sample(sample, HORIZON, 'input', time_index=hrz_idxs)
        self._add_to_sample(sample, HORIZON, 'target', time_index=hrz_idxs)
        self._add_to_sample(sample, HORIZON, 'auxiliary', time_index=hrz_idxs)

        # get static data
        self._add_to_sample(sample, STATIC, 'input')
        self._add_to_sample(sample, STATIC, 'target')
        self._add_to_sample(sample, STATIC, 'auxiliary')

        # get connectivity
        if self.edge_index is not None:
            sample.input['edge_index'] = self.edge_index
            if self.edge_weight is not None:
                sample.input['edge_weight'] = self.edge_weight
        return sample
    
    
    def get_tensor(self, key: str, preprocess: bool = False,
                   time_index: Union[List, Tensor] = None,
                   node_index: Union[List, Tensor] = None) \
            -> Tuple[Tensor, Optional[ScalerModule]]:
        # get dataset item
        if key not in self.keys:
            raise KeyError(f"{key} not in dataset {self.name}.")

        # convert indices
        time_index = self._get_time_index(time_index, layout='index')
        node_index = self._get_time_index(node_index, layout='index')
        x = take(getattr(self, key),
                 self.patterns[key],
                 backend=torch,
                 time_index=time_index,
                 node_index=node_index)

        # get scaler (if any)
        scaler = None
        if key in self.scalers is not None:
            scaler = self.scalers[key].slice(time_index=time_index,
                                             node_index=node_index)
            if preprocess:  # transform tensor
                x = scaler.transform(x)
        return x, scaler
    
    def __repr__(self):
        return "{}(n_samples={}, n_nodes={}, n_channels={})" \
            .format(self.name, self.n_samples, self.n_nodes, self.n_channels)

    def __getitem__(self, item) -> Data:
        if isinstance(item, int) and item < 0:
            # compute item's actual index
            item = self._indices.size(0) + item
        else:
            # convert slice to indexes
            item = parse_index(item, length=self.n_samples, layout='index')
        item = self.get(item)
        if self.transform is not None:
            item = self.transform(item)
        return item

    def __contains__(self, item) -> bool:
        return item in self.keys

    def __len__(self) -> int:
        return self.n_samples

    def __getattr__(self, item):
        if '_covariates' in self.__dict__ and item in self._covariates:
            return self._covariates[item]['value']
        if 'input_map' in self.__dict__ and item in self.input_map:
            return self.collate_item_elem(item)[0]
        raise AttributeError(f"'{self.__class__.__name__}' object has no "
                             f"attribute '{item}'")

    def __setattr__(self, key, value):
        super(SpatioTemporalDataset, self).__setattr__(key, value)
        if key in _WINDOWING_['window'] and \
                all([hasattr(self, attr) for attr in _WINDOWING_['window']]):
            self._window_range = torch.arange(0, self.window, self.window_lag)
        if key in _WINDOWING_['horizon'] and \
                all([hasattr(self, attr) for attr in _WINDOWING_['horizon']]):
            self._horizon_range = torch.arange(
                self.horizon_offset, self.horizon_offset + self.horizon,
                self.horizon_lag)
        if key in _WINDOWING_['indices'] and \
                all([hasattr(self, attr) for attr in _WINDOWING_['indices']]):
            self._indices = torch.arange(0,
                                         self.n_steps - self.sample_span + 1,
                                         self.stride)

    def __delattr__(self, item):
        if item in _WINDOWING_['all']:
            raise AttributeError(f"Cannot delete attribute '{item}'.")
        elif item == 'mask':
            self.set_mask(None)
        elif item == 'trend':
            self.set_trend(None)
        elif item in self._covariates:
            self.remove_covariate(item)
        else:
            super(SpatioTemporalDataset, self).__delattr__(item)

    # Dataset properties ######################################################


    @property
    def n_edges(self) -> Optional[int]:
        """Number of edges in the dataset, if a connectivity is set."""
        if isinstance(self.edge_index, Tensor):
            return self.edge_index.size(1)
        elif isinstance(self.edge_index, SparseTensor):
            return self.edge_index.numel()
        return None


    @property
    def patterns(self) -> dict:
        """Shows the dimension of dataset's tensors in a more informative way.

        The pattern mapping can be useful to glimpse on how data are arranged.
        The convention we use is the following:

        * 't' stands for “number of time steps”
        * 'n' stands for “number of nodes”
        * 'f' stands for “number of features” (per node)
        * 'e' stands for “number edges”
        """
        patterns = {'target': 't n f'}
        # add mask pattern
        if self.mask is not None:
            patterns['mask'] = 't n f'
        # add connectivity patterns
        if self.edge_index is not None:
            patterns['edge_index'] = '2 e' if isinstance(
                self.edge_index, Tensor) else 'n n'
            if self.edge_weight is not None:
                patterns['edge_weight'] = 'e'
        # add covariates patterns
        patterns.update(
            {name: attr['pattern']
             for name, attr in self._covariates.items()})
        return patterns

    @property
    def batch_patterns(self) -> dict:
        """Shows the dimension of dataset's tensors in a more informative way.

        The pattern mapping can be useful to glimpse on how data are arranged.
        The convention we use is the following:

        * 't' stands for “number of time steps”
        * 'n' stands for “number of nodes”
        * 'f' stands for “number of features” (per node)
        * 'e' stands for “number edges”
        """
        # add input map patterns
        patterns = {
            name: attr.pattern
            for name, attr in self.input_map.items()
        }
        # add mask pattern
        if self.mask is not None:
            patterns['mask'] = 't n f'
        # add connectivity patterns
        if self.edge_index is not None:
            patterns['edge_index'] = '2 e' if isinstance(
                self.edge_index, Tensor) else 'n n'
            if self.edge_weight is not None:
                patterns['edge_weight'] = 'e'
        # add target map patterns
        patterns.update(
            {name: attr.pattern
             for name, attr in self.target_map.items()})
        return patterns

    @property
    def keys(self) -> list:
        """Keys in dataset."""
        return list(self.patterns.keys())

    @property
    def batch_keys(self) -> list:
        """Keys in dataset item."""
        return list(self.batch_patterns.keys())

    # Indexing

    @property
    def horizon_offset(self) -> int:
        """Lag of starting step of horizon."""
        return self.window + self.delay

    @property
    def sample_span(self) -> int:
        """Total number of steps of an item, including window and horizon."""
        return max(self.horizon_offset + self.horizon, self.window)

    @property
    def samples_offset(self) -> int:
        """Difference (in number of steps) between two items."""
        return int(np.ceil(self.window / self.stride))

    @property
    def indices(self) -> Tensor:
        """Indices of the dataset. The :obj:`i`-th item is mapped to
        :obj:`indices[i]`"""
        return self._indices

    @property
    def n_samples(self) -> int:
        """Number of samples (i.e., items) in the dataset."""
        return len(self._indices)

    # Covariates properties

    @property
    def covariates(self) -> dict:
        """Mapping of dataset's covariates."""
        return {name: attr['value'] for name, attr in self._covariates.items()}

    @property
    def exogenous(self) -> dict:
        """Time-varying covariates of the dataset's target."""
        return {
            name: attr['value']
            for name, attr in self._covariates.items()
            if 't' in attr['pattern']
        }

    @property
    def attributes(self) -> dict:
        """Static features related to the dataset."""
        return {
            name: attr['value']
            for name, attr in self._covariates.items()
            if 't' not in attr['pattern']
        }

    @property
    def n_covariates(self) -> int:
        """Number of covariates in the dataset."""
        return len(self._covariates)

    # flags

    @property
    def has_connectivity(self) -> bool:
        """Whether the dataset has a connectivity."""
        return self.edge_index is not None

    @property
    def has_mask(self) -> bool:
        """Whether the dataset has a mask denoting valid values in target."""
        return self.mask is not None

    @property
    def has_covariates(self) -> bool:
        """Whether the dataset has covariates to the target tensor."""
        return self.n_covariates > 0

    # Map Dataset to item #####################################################

    @property
    def targets(self) -> BatchMap:
        return self.target_map

    def reset_input_map(self):
        self._clear_batch_map('input')
        self.input_map['x'] = BatchMapItem('target',
                                           SynchMode.WINDOW,
                                           preprocess=True,
                                           cat_dim=None,
                                           pattern='t n f',
                                           shape=self.shape)
        for name, attr in self._covariates.items():
            self.input_map[name] = BatchMapItem(name,
                                                SynchMode.WINDOW,
                                                preprocess=True,
                                                cat_dim=None,
                                                pattern=attr['pattern'],
                                                shape=attr['value'].size())

    def reset_target_map(self):
        self._clear_batch_map('target')
        self.target_map['y'] = BatchMapItem('target',
                                            SynchMode.HORIZON,
                                            preprocess=False,
                                            cat_dim=None,
                                            pattern='t n f',
                                            shape=self.shape)

    def reset_auxiliary_map(self):
        self._clear_batch_map('auxiliary')
        if self.mask is not None:
            self.auxiliary_map['mask'] = BatchMapItem('mask',
                                                      SynchMode.HORIZON,
                                                      preprocess=False,
                                                      cat_dim=None,
                                                      pattern='t n f',
                                                      shape=self.mask.shape)

    def set_input_map(self, input_map=None, **kwargs):
        self._clear_batch_map('input')
        self.update_input_map(input_map, **kwargs)

    def set_target_map(self, target_map=None, **kwargs):
        self._clear_batch_map('target')
        self.update_target_map(target_map, **kwargs)

    def set_auxiliary_map(self, auxiliary_map=None, **kwargs):
        self._clear_batch_map('auxiliary')
        self.update_auxiliary_map(auxiliary_map, **kwargs)

    def update_input_map(self, input_map=None, **kwargs):
        self._update_batch_map('input', input_map, **kwargs)

    def update_target_map(self, target_map=None, **kwargs):
        self._update_batch_map('target', target_map, **kwargs)

    def update_auxiliary_map(self, auxiliary_map=None, **kwargs):
        self._update_batch_map('auxiliary', auxiliary_map, **kwargs)

    def _clear_batch_map(self, endpoint):
        batch_map = getattr(self, f"{endpoint}_map")
        for key in batch_map:
            if key in self._batch_scalers:
                del self._batch_scalers[key]
        setattr(self, f"{endpoint}_map", BatchMap())

    def _update_batch_map(self, endpoint, batch_map=None, **kwargs):
        # check batch map
        if batch_map is None:
            batch_map = BatchMap()
        elif not isinstance(batch_map, Mapping):
            raise TypeError(f"Type {type(batch_map)} is not valid for "
                            f"`{endpoint}_map` (must be a mapping).")
        # update from kwargs
        batch_map.update(**kwargs)
        # update endpoint_map
        endpoint_map = getattr(self, f"{endpoint}_map")
        endpoint_map.update(**batch_map)
        # update 'pattern' and 'shape' to added/updated keys
        for name in batch_map:
            item: BatchMapItem = endpoint_map[name]
            # keys sanity check and compute pattern and shape
            keys = item.keys
            if len(keys) > 1:
                tensor, scaler, pattern = self.collate_keys(
                    keys, cat_dim=item.cat_dim, return_pattern=True)
                item.shape, item.pattern = tuple(tensor.size()), pattern
                if scaler is not None:
                    self._batch_scalers[name] = scaler
            else:
                item.shape = tuple(getattr(self, keys[0]).size())
                item.pattern = self.patterns[keys[0]]

    # Getters #################################################################


    def expand_scaler(self, key: str, pattern: Optional[str] = None,
                      time_index: Union[List, Tensor] = None,
                      node_index: Union[List, Tensor] = None) \
            -> Optional[ScalerModule]:
        # check if there is a scaler
        if key not in self.keys:
            raise KeyError(f"{key} not in {self.name}.")
        elif key not in self.scalers:
            return None
        # convert indices
        time_index = self._get_time_index(time_index, layout='index')
        node_index = self._get_time_index(node_index, layout='index')
        # get params
        if pattern is None:
            return self.scalers[key]
        # if there is an out-pattern, create new scaler
        scaler = ScalerModule(self.scalers[key], pattern=pattern)
        pattern = self.patterns[key] + ' -> ' + pattern
        scaler.bias = broadcast(scaler.bias,
                                pattern,
                                backend=torch,
                                time_index=time_index,
                                node_index=node_index)
        scaler.scale = broadcast(scaler.scale,
                                 pattern,
                                 backend=torch,
                                 time_index=time_index,
                                 node_index=node_index)
        return scaler

    def expand_tensor(self,
                      key: str,
                      pattern: str,
                      time_index: Union[List, Tensor] = None,
                      node_index: Union[List, Tensor] = None):
        x = getattr(self, key)
        pattern = self.patterns[key] + ' -> ' + pattern
        x = broadcast(x,
                      pattern,
                      t=self.n_steps,
                      n=self.n_nodes,
                      backend=torch,
                      time_index=time_index,
                      node_index=node_index)
        return x


    def collate_item_elem(self, key: str,
                          time_index: Union[List, Tensor] = None,
                          node_index: Union[List, Tensor] = None) \
            -> Tuple[Tensor, Optional[ScalerModule]]:
        # get batch item
        if key in self.input_map:
            itm = self.input_map[key]
        elif key in self.target_map:
            itm = self.target_map[key]
        else:
            raise KeyError(f"{key} not in any batch map of {self.name}.")

        # expand and concatenate tensors
        x = torch.cat([
            self.expand_tensor(k, itm.pattern, time_index, node_index)
            for k in itm.keys
        ],
                      dim=itm.cat_dim)

        # get scaler (if any)
        scaler = None
        if key in self._batch_scalers:
            scaler = self._batch_scalers[key].slice(time_index=time_index,
                                                    node_index=node_index)
            if itm.preprocess:  # transform tensor
                x = scaler.transform(x)
        return x, scaler

    def collate_keys(self,
                     keys: Iterable,
                     preprocess: bool = False,
                     time_index: Union[List, Tensor] = None,
                     node_index: Union[List, Tensor] = None,
                     cat_dim: Optional[int] = None,
                     return_pattern: bool = False):
        if any([key not in self.keys for key in keys]):
            unmatch = set(keys).difference(self.keys)
            raise KeyError(f"{unmatch} not in {self.name}.")
        pattern = outer_pattern([self.patterns[key] for key in keys])
        tensors, scalers = list(), list()
        for key in keys:
            tensor = self.expand_tensor(key, pattern, time_index, node_index)
            scaler = self.expand_scaler(key, pattern, time_index, node_index)
            if preprocess and scaler is not None:
                tensor = scaler(tensor)
            tensors.append(tensor)
            scalers.append(scaler)
        if len(tensors) == 1:
            if return_pattern:
                return tensors[0], scalers[0], pattern
            return tensors[0], scalers[0]
        if cat_dim is not None:
            scalers = ScalerModule.cat(scalers,
                                       dim=cat_dim,
                                       sizes=[t.size() for t in tensors])
            tensors = torch.cat(tensors, dim=cat_dim)
        if return_pattern:
            return tensors, scalers, pattern
        return tensors, scalers

    def get_mask(self, dtype: Union[type, str, np.dtype] = None) -> Tensor:
        mask = self.mask if self.has_mask else ~torch.isnan(self.target)
        if dtype is not None:
            assert dtype in ['bool', 'uint8', bool, torch.bool, torch.uint8]
            mask = mask.to(dtype)
        return mask

    # Getters helpers

    def _get_time_index(self,
                        time_index: IndexSlice = None,
                        layout: Literal['index', 'slice', 'mask'] = 'index'):
        return parse_index(time_index, length=self.n_steps, layout=layout)

    def _get_node_index(self,
                        node_index: IndexSlice = None,
                        layout: Literal['index', 'slice', 'mask'] = 'index'):
        return parse_index(node_index, length=self.n_nodes, layout=layout)

    def _add_to_sample(self,
                       out,
                       synch_mode,
                       endpoint='input',
                       time_index=None,
                       node_index=None):
        batch_map = getattr(self, f"{endpoint}_map")
        for key, item in batch_map.by_synch_mode(synch_mode).items():
            if len(item.keys) > 1:
                tensor, scaler = self.collate_item_elem(key,
                                                        time_index=time_index,
                                                        node_index=node_index)
            else:
                tensor, scaler = self.get_tensor(item.keys[0],
                                                 preprocess=item.preprocess,
                                                 time_index=time_index,
                                                 node_index=node_index)
            if endpoint == 'auxiliary':
                out[key] = tensor
            else:
                getattr(out, endpoint)[key] = tensor
            if scaler is not None:
                out.transform[key] = scaler

    # Setters #################################################################

    def set_data(self, data: DataArray):
        r"""Set sequence of primary channels at :obj:`self.data`."""
        self.target = self._parse_target(data)

    def set_mask(self,
                 mask: Optional[DataArray],
                 add_to_auxiliary_map: bool = True):
        r"""Set mask of target channels, i.e., a bool for each (node, time
        step, channel) triplet denoting if corresponding value in target is
        observed (obj:`True`) or not (obj:`False`)."""
        if mask is not None:
            mask = self._parse_target(mask)
            if mask.dtype not in [torch.bool, torch.uint8]:
                raise RuntimeError("Mask is not boolean, accepted types are "
                                   f"{[torch.bool, torch.uint8]}.")
            # check mask length is equal to target's length
            self._check_same_dim(mask.size(0), 'n_steps', 'mask')
            # check mask nodes/features are broadcastable to
            # target's nodes/features
            self._check_same_dim(mask.size(1),
                                 'n_nodes',
                                 'mask',
                                 allow_broadcasting=True)
            self._check_same_dim(mask.size(-1),
                                 'n_channels',
                                 'mask',
                                 allow_broadcasting=True)
            if add_to_auxiliary_map:
                self.auxiliary_map['mask'] = BatchMapItem('mask',
                                                          SynchMode.HORIZON,
                                                          preprocess=False,
                                                          cat_dim=None,
                                                          pattern='t n f',
                                                          shape=mask.shape)
        self.mask = mask

    def set_connectivity(self,
                         connectivity: Union[SparseTensArray,
                                             Tuple[DataArray]],
                         target_layout: Optional[str] = None):
        r"""Set dataset connectivity.

        The input can be either a
        dense/sparse matrix :math:`\mathbf{A} \in \mathbb{R}^{N \times N}`
        or an (:obj:`edge_index` :math:`\in \mathbb{N}^{2 \times E}`,
        :obj:`edge_weight` :math:`\in \mathbb{R}^{E})` tuple. If
        :obj:`target_layout` is :obj:`None`, the input layout will be
        preserved (e.g., a sparse matrix will be stored as a
        :class:`torch_sparse.SparseTensor`), otherwise the connectivity is
        converted to the specified layout. In any case, the connectivity
        will be stored in the attribute :obj:`edge_index`, and the weights
        will be eventually stored as :obj:`edge_weight`.

        Args:
            connectivity (SparseTensArray, tuple, optional): The connectivity
            target_layout (str, optional): If specified, the input connectivity
                is converted to this layout. Possible options are [dense,
                sparse, edge_index]. If :obj:`None`, the target layout is
                inferred from the input.
                (default: :obj:`None`)
        """
        self.edge_index, self.edge_weight = self._parse_connectivity(
            connectivity, target_layout)

    # Setter for covariates

    def add_covariate(self,
                      name: str,
                      value: DataArray,
                      pattern: Optional[str] = None,
                      add_to_input_map: bool = True,
                      synch_mode: Optional[SynchMode] = None,
                      preprocess: bool = True,
                      convert_precision: bool = True):
        r"""Add covariate to the dataset. Examples of covariate are
        exogenous signals (in the form of dynamic multidimensional data) or
        static attributes (e.g., graph/node metadata). Parameter :obj:`pattern`
        specifies what each axis refers to:

        - 't': temporal dimension;
        - 'n': node dimension;
        - 'c'/'f': channels/features dimension.

        For instance, the pattern of a node-level covariate is 't n f', while a
        pairwise metric between nodes has pattern 'n n'.

        Args:
            name (str): the name of the object. You can then access the added
                object as :obj:`dataset.{name}`.
            value (DataArray): the object to be added. Can be a
                :class:`~pandas.DataFrame`, a :class:`~numpy.ndarray` or a
                :class:`~torch.Tensor`.
            pattern (str, optional): the pattern of the object. A pattern
                specifies what each axis refers to:

                - 't': temporal dimension;
                - 'n': node dimension;
                - 'c'/'f': channels/features dimension.

                If :obj:`None`, the pattern is inferred from the shape.
                (default :obj:`None`)
            add_to_input_map (bool): Whether to map the covariate to dataset
                item when calling :obj:`get` methods.
                (default: :obj:`True`)
            synch_mode (SynchMode): How to synchronize the exogenous variable
                inside dataset item, i.e., with the window slice
                (:obj:`SynchMode.WINDOW`) or horizon (:obj:`SynchMode.HORIZON`).
                It applies only for time-varying covariates.
                (default: :obj:`SynchMode.WINDOW`)
            preprocess (bool): If :obj:`True` and the dataset has a scaler with
                same key, then data are scaled when calling :obj:`get` methods.
                (default: :obj:`True`)
            convert_precision (bool): If :obj:`True`, then cast :attr:`value`
                with dataset's precision.
                (default: :obj:`True`)
        """
        # validate name. name cannot be an attribute of self, but allow override
        self._check_name(name)
        value, pattern = self._parse_covariate(
            value, pattern, name=name, convert_precision=convert_precision)
        self._covariates[name] = dict(value=value, pattern=pattern)
        if add_to_input_map:
            self.input_map[name] = BatchMapItem(name,
                                                synch_mode,
                                                preprocess,
                                                cat_dim=None,
                                                pattern=pattern,
                                                shape=value.size())

    def update_covariate(self,
                         name: str,
                         value: Optional[DataArray] = None,
                         pattern: Optional[str] = None,
                         add_to_input_map: bool = True,
                         synch_mode: Optional[SynchMode] = None,
                         preprocess: bool = None):
        r"""Update a covariate already in the dataset.

        Args:
            name (str): the name of the object. You can then access the added
                object as :obj:`dataset.{name}`.
            value (DataArray, optional): the object to be added. Can be a
                :class:`~pandas.DataFrame`, a :class:`~numpy.ndarray` or a
                :class:`~torch.Tensor`.
            pattern (str, optional): the pattern of the object. A pattern
                specifies what each axis refers to:

                - 't': temporal dimension;
                - 'n': node dimension;
                - 'c'/'f': channels/features dimension.

                If :obj:`None`, the pattern is inferred from the shape.
                (default :obj:`None`)
            add_to_input_map (bool): Whether to map the covariate to dataset
                item when calling :obj:`get` methods.
                (default: :obj:`True`)
            synch_mode (SynchMode): How to synchronize the exogenous variable
                inside dataset item, i.e., with the window slice
                (:obj:`SynchMode.WINDOW`) or horizon (:obj:`SynchMode.HORIZON`).
                It applies only for time-varying covariates.
                (default: :obj:`SynchMode.WINDOW`)
            preprocess (bool): If :obj:`True` and the dataset has a scaler with
                same key, then data are scaled when calling :obj:`get` methods.
                (default: :obj:`True`)
        """
        # validate name. name cannot be an attribute of self, but allow override
        if name not in self._covariates:
            raise RuntimeError(f"There is not a covariate named {name} in "
                               f"{self.__class__.__name__}")
        # override value (with or without explicit pattern)
        if value is not None:
            value, pattern = self._parse_covariate(value, pattern, name=name)
        # override pattern, rearranging the stored covariate
        elif pattern is not None:
            pattern = check_pattern(pattern, ndim=value.ndim)
            from einops import rearrange
            old_value = self._covariates[name]['value']
            old_pattern = self._covariates[name]['pattern']
            value = rearrange(old_value, f'{old_pattern} -> {pattern}')
        # update the covariate
        self._covariates[name] = dict(value=value, pattern=pattern)
        # update input map
        if add_to_input_map:
            if name in self.input_map:
                kwargs = dict(synch_mode=self.input_map[name].synch_mode,
                              preprocess=self.input_map[name].preprocess)
            else:
                kwargs = dict(synch_mode=None, preprocess=True)
            if preprocess is not None:
                kwargs['preprocess'] = preprocess
            if synch_mode is not None:
                kwargs['synch_mode'] = synch_mode
            shape = tuple(self._covariates[name]['value'].size())
            pattern = self._covariates[name]['pattern']
            self.input_map[name] = BatchMapItem(name,
                                                **kwargs,
                                                cat_dim=None,
                                                pattern=pattern,
                                                shape=shape)

    def remove_covariate(self, name: str):
        r"""Delete covariate from the dataset.

        Args:
            name (str): the name of the covariate to be deleted.
        """
        try:
            # remove covariate
            del self._covariates[name]
            # remove associated scaler
            if name in self.scalers:
                del self.scalers[name]
            # ATTENTION! remove entirely map item with covariate in keys
            for _map in [self.input_map, self.target_map, self.auxiliary_map]:
                for _map_item in _map:
                    if name in _map_item.keys:
                        del _map[_map_item]
        except Exception as e:
            raise e

    def add_exogenous(self,
                      name: str,
                      value: DataArray,
                      node_level: bool = True,
                      add_to_input_map: bool = True,
                      synch_mode: SynchMode = WINDOW,
                      preprocess: bool = True):
        r"""Shortcut method to add a time-varying covariate.

        Exogenous variables are time-varying covariates of the dataset's target.
        They can either be graph-level (i.e., with same temporal
        length as :obj:`target` but with no node dimension) or node-level (i.e.,
        with same temporal and node size as :obj:`target`).

        Args:
            name (str): The name of the exogenous variable. If the name starts
                with :obj:`"global_"`, the variable is assumed to be graph-level
                (overriding parameter :obj:`node_level`), and the
                :obj:`"global_"` prefix is removed from the name.
            value (DataArray): The data sequence. Can be a
                :class:`~pandas.DataFrame`, a :class:`~numpy.ndarray` or a
                :class:`~torch.Tensor`.
            node_level (bool): Whether the input variable is node- or
                graph-level. If a 2-dimensional array is given and node-level is
                :obj:`True`, it is assumed that the input has one channel.
                (default: :obj:`True`)
            add_to_input_map (bool): Whether to map the exogenous variable to
                dataset item when calling :obj:`get` methods.
                (default: :obj:`True`)
            synch_mode (SynchMode): How to synchronize the exogenous variable
                inside dataset item, i.e., with the window slice
                (:obj:`SynchMode.WINDOW`) or horizon (:obj:`SynchMode.HORIZON`).
                (default: :obj:`SynchMode.WINDOW`)
            preprocess (bool): If :obj:`True` and the dataset has a scaler with
                same key, then data are scaled when calling :obj:`get` methods.
                (default: :obj:`True`)

        Returns:
            SpatioTemporalDataset: the dataset with added exogenous.
        """
        if name.startswith('global_'):
            name = name[7:]
            node_level = False
        pattern = 't n f' if node_level else 't f'
        self.add_covariate(name,
                           value,
                           pattern,
                           synch_mode=synch_mode,
                           add_to_input_map=add_to_input_map,
                           preprocess=preprocess)

    # Setters for preprocessing

    def set_trend(self, trend: Optional[DataArray]):
        r"""Set trend of dataset's target data."""
        if trend is not None:
            trend = self._parse_target(trend)
            # check trend length is equal to target's length
            self._check_same_dim(trend.size(0), 'n_steps', 'mask')
            # check trend nodes/features are broadcastable to
            # target's nodes/features
            self._check_same_dim(trend.size(1),
                                 'n_nodes',
                                 'mask',
                                 allow_broadcasting=True)
            self._check_same_dim(trend.size(-1),
                                 'n_channels',
                                 'mask',
                                 allow_broadcasting=True)
            if 'target' in self.scalers:
                if self.__target_bias is None:
                    self.__target_bias = self.scalers['target'].bias
                self.scalers['target'].bias = trend
        else:
            if 'target' in self.scalers:
                self.scalers['target'].bias = self.__target_bias
                # refresh maps
                self.update_input_map(self.input_map)
                self.update_target_map(self.target_map)
        self.trend = trend

    def add_scaler(self, key: str, scaler: Union[Scaler, ScalerModule]):
        r"""Add a :class:`tsl.data.preprocessing.Scaler` for the object indexed
        by :obj:`key` in the dataset.

        Args:
            key (str): The name of the variable associated to the scaler. It
                must be a temporal variable, i.e., :obj:`data` or an exogenous.
            scaler (Scaler): The :class:`~tsl.data.preprocessing.Scaler`.
        """
        if key not in self.keys:
            raise KeyError(f"{key} not in {self.name}.")
        # copy to ScalerModule
        scaler = ScalerModule(scaler)
        pattern = self.patterns[key]
        self._check_pattern(scaler.bias,
                            pattern,
                            name=f"scaler ({key})",
                            allow_broadcasting=True)
        self._check_pattern(scaler.scale,
                            pattern,
                            name=f"scaler ({key})",
                            allow_broadcasting=True)
        if key == 'target' and self.trend is not None:
            self.__target_bias = scaler.bias
            scaler.bias = scaler.bias + self.trend
        scaler.pattern = pattern
        self.scalers[key] = scaler
        # cache batch scaler if target tensor is in a multi-key batch item
        for bm in [self.input_map, self.target_map, self.auxiliary_map]:
            for bm_key, bm_item in bm.items():
                if key in bm_item.keys and len(bm_item.keys) > 1:
                    tensor, scaler = self.collate_keys(bm_item.keys,
                                                       cat_dim=bm_item.cat_dim,
                                                       return_pattern=False)
                    self._batch_scalers[bm_key] = scaler

    # Dataset trimming ########################################################

    def reduce(self,
               time_index: Optional[IndexSlice] = None,
               node_index: Optional[IndexSlice] = None):
        """Reduce the dataset in terms of number of steps and nodes. Returns a
        copy of the reduced dataset.

        If dataset has a connectivity, edges ending to or starting from removed
        nodes will be removed as well.

        Args:
            time_index (IndexSlice, optional): index or mask of the nodes to
                keep after reduction.
                (default: :obj:`None`)
            node_index (IndexSlice, optional): index or mask of the nodes to
                keep after reduction.
                (default: :obj:`None`)
        """
        return deepcopy(self).reduce_(time_index, node_index)

    def reduce_(self,
                time_index: Optional[IndexSlice] = None,
                node_index: Optional[IndexSlice] = None):
        """Reduce the dataset in terms of number of steps and nodes. This is an
        inplace operation.

        If dataset has a connectivity, edges ending to or starting from removed
        nodes will be removed as well.

        Args:
            time_index (IndexSlice, optional): index or mask of the nodes to
                keep after reduction.
                (default: :obj:`None`)
            node_index (IndexSlice, optional): index or mask of the nodes to
                keep after reduction.
                (default: :obj:`None`)
        """
        # use slice to reduce known tensor
        time_slice = self._get_time_index(time_index, layout='slice')
        node_slice = self._get_node_index(node_index, layout='slice')
        # use index to reduce using index-fed functions
        time_index = self._get_time_index(time_index, layout='index')
        node_index = self._get_node_index(node_index, layout='index')
        try:
            if self.edge_index is not None and node_index is not None:
                self.edge_index, self.edge_weight = reduce_graph(
                    node_index,
                    self.edge_index,
                    self.edge_weight,
                    num_nodes=self.n_nodes)
            self.target = self.target[time_slice, node_slice]
            if self.index is not None:
                self.index = self.index[time_index.numpy()]
            if self.mask is not None:
                self.mask = self.mask[time_slice, node_slice]
            if self.trend is not None:
                self.trend = self.trend[time_slice, node_slice]
            for name, attr in self._covariates.items():
                x, scaler = self.get_tensor(name,
                                            time_index=time_index,
                                            node_index=node_index)
                attr['value'] = x
                if scaler is not None:
                    self.scalers[name] = scaler
        except Exception as e:
            raise e
        return self

    @contextmanager
    def change_windowing(self, **kwargs):
        default, indices = dict(), self._indices
        try:
            assert all([k in _WINDOWING_['all'][1:] for k in kwargs])
            for k, v in kwargs.items():
                default[k] = getattr(self, k)
                setattr(self, k, v)
            yield self
        finally:
            for k, v in default.items():
                setattr(self, k, v)
            self._indices = indices

    # Indexing ################################################################

    def get_window_indices(self, item):
        idx = self._indices[item]
        if idx.dim():  # idx is list of indices
            return idx[:, None] + self._window_range[None]
        else:  # idx is one index
            return idx + self._window_range

    def get_horizon_indices(self, item):
        idx = self._indices[item]
        if idx.dim():  # idx is list of indices
            return idx[:, None] + self._horizon_range[None]
        else:  # idx is one index
            return idx + self._horizon_range

    def set_indices(self, indices: TensArray):
        indices = torch.as_tensor(indices, dtype=torch.long)
        max_index = self.n_steps - self.sample_span
        assert all((indices >= 0) & (indices <= max_index)), \
            f"indices must be in the range [0, {max_index}] for {self.name}."
        self._indices = indices

    def expand_indices(self, indices=None, unique=False, merge=False):
        indices = torch.arange(self.n_samples) if indices is None else indices

        ds_indices = dict()
        if self.window > 0:
            ds_indices['window'] = self.get_window_indices(indices)
        ds_indices['horizon'] = self.get_horizon_indices(indices)

        if merge:
            return torch.unique(
                torch.cat(
                    [v.contiguous().view(-1) for v in ds_indices.values()]))

        if unique:
            ds_indices = {k: torch.unique(v) for k, v in ds_indices.items()}

        return ds_indices

    def overlapping_indices(self,
                            idxs1,
                            idxs2,
                            synch_mode: Union[SynchMode, str] = WINDOW,
                            as_mask=False):
        if isinstance(synch_mode, SynchMode):
            synch_mode = synch_mode.name
        idxs1, idxs2 = np.asarray(idxs1), np.asarray(idxs2)
        ts1 = self.expand_indices(idxs1)[synch_mode.lower()].numpy()
        ts2 = self.expand_indices(idxs2)[synch_mode.lower()].numpy()
        common_ts = np.intersect1d(ts1, ts2)

        def is_overlapping(sample):
            return np.any(np.in1d(sample, common_ts))

        m1 = np.apply_along_axis(is_overlapping, 1, ts1)
        m2 = np.apply_along_axis(is_overlapping, 1, ts2)
        if as_mask:
            return m1, m2
        return idxs1[m1], idxs2[m2]

    def data_timestamps(self, indices=None, unique=False) -> Optional[dict]:
        if self.index is None:
            return None
        ds_indices = self.expand_indices(indices, unique=unique)
        index = self.index if unique else self.index.to_numpy()
        ds_timestamps = {k: index[v.numpy()] for k, v in ds_indices.items()}
        return ds_timestamps

    # Representation ##########################################################

    def numpy(self):
        return np.asarray(self.target)

    def dataframe(self):
        columns = pd.MultiIndex.from_product(
            [range(self.n_nodes), range(self.n_channels)],
            names=['nodes', 'channels'])
        data = self.numpy().reshape((-1, self.n_nodes * self.n_channels))
        return pd.DataFrame(data=data, index=self.index, columns=columns)

    # Utilities ###############################################################

    def save(self, filename: str) -> None:
        """Save :obj:`SpatioTemporalDataset` to disk.

        Args:
            filename (str): path to filename for storage.
        """
        torch.save(self, filename)

    @classmethod
    def load(cls, filename: str) -> "SpatioTemporalDataset":
        """Load instance of :obj:`SpatioTemporalDataset` from disk.

        Args:
            filename (str): path of :obj:`SpatioTemporalDataset`.
        """
        obj = torch.load(filename)
        if not isinstance(obj, cls):
            raise TypeError(f"Loaded file is not of class {cls}.")
        return obj

    @classmethod
    def from_dataset(cls,
                     dataset,
                     connectivity: Optional[Union[SparseTensArray,
                                                  Tuple[DataArray]]] = None,
                     covariate_keys: List[str] = None,
                     input_map: Optional[Union[Mapping, BatchMap]] = None,
                     target_map: Optional[Union[Mapping, BatchMap]] = None,
                     auxiliary_map: Optional[Union[Mapping, BatchMap]] = None,
                     scalers: Optional[Mapping[str, Scaler]] = None,
                     trend: Optional[DataArray] = None,
                     window: int = 12,
                     horizon: int = 1,
                     delay: int = 0,
                     stride: int = 1,
                     window_lag: int = 1,
                     horizon_lag: int = 1) -> "SpatioTemporalDataset":
        """Create a :class:`~tsl.data.SpatioTemporalDataset` from a
        :class:`~tsl.datasets.prototypes.TabularDataset`.
        """
        covariates = dataset._covariates
        if covariate_keys is not None:
            covariates = {k: v for k, v in covariates if k in covariate_keys}
        return cls(target=dataset.target,
                   index=dataset.index,
                   mask=dataset.mask,
                   covariates=covariates,
                   name=dataset.name,
                   precision=dataset.precision,
                   connectivity=connectivity,
                   input_map=input_map,
                   target_map=target_map,
                   auxiliary_map=auxiliary_map,
                   scalers=scalers,
                   trend=trend,
                   window=window,
                   horizon=horizon,
                   delay=delay,
                   stride=stride,
                   window_lag=window_lag,
                   horizon_lag=horizon_lag)


NameError: name 'Tensor' is not defined

In [10]:
# d = train_loader.get_data()
# # x[time_index].bfill(dim='dim_0').load().data
# d[:10].load().data

In [11]:
# from tsl.data import SpatioTemporalDataset
import pandas as pd
data = train_loader.get_data()
# time_index = pd.DatetimeIndex(data.time,freq='infer')
from tsl.data.preprocessing import StandardScaler, Scaler, ScalerModule, MinMaxScaler
# ScalerModule(scaler=MinMaxScaler())

torch_dataset = SpatioTemporalDataset(target=train_loader.get_data(),
                                      connectivity=train_loader.get_connectivity(),
                                    #   index=time_index,
                                      horizon=40,
                                      window=112,
                                      stride=1
                                      )
# test_dataset = SpatioTemporalDataset(target=test_loader.get_data(),
#                                       connectivity=test_loader.get_connectivity(),
#                                       horizon=40,
#                                       window=112,
#                                       stride=1)
validation_dataset = SpatioTemporalDataset(target=validation_loader.get_data(),
                                      connectivity=validation_loader.get_connectivity(),
                                      horizon=40,
                                      window=112,
                                      stride=1)

In [12]:
# from tsl.data.preprocessing import Scaler
# from tsl.typing import TensArray

# class WeatherNormalizationScaler(Scaler):
#     def fit(self, x: TensArray, *args, **kwargs):
#         return self

In [13]:
# from tsl.data.preprocessing import Scaler
# from tsl.typing import TensArray

# class WeatherNormalizationScaler(Scaler):
#     def fit(self, x: TensArray, *args, **kwargs):
#         return self
# scalers = {
#     'target': WeatherNormalizationScaler(bias, scale)
# }

In [14]:
from tsl.data.loader import StaticGraphLoader
# from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader

stg_train = StaticGraphLoader(torch_dataset, batch_size=5, num_workers=10,  persistent_workers=True, prefetch_factor=16)
stg_valid = StaticGraphLoader(validation_dataset, batch_size=5, num_workers=10, persistent_workers=True, prefetch_factor=16)

In [15]:
# torch_dataset.__getitem__(0)

In [16]:
# tsl staticgraph loader
# next(iter(stg_train))

In [17]:
# pyg dataloader
# next(iter(stg_valid))

In [18]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter, AtTimeStepSplitter)
from tsl.data.preprocessing import StandardScaler, Scaler, ScalerModule, MinMaxScaler
import datetime
# from tsl.data.preprocessing import StandardScaler, Scaler, ScalerModule, MinMaxScaler
# ScalerModule(scaler=MinMaxScaler())
# Normalize data using mean and std computed over time and node dimensions
# scalers = {'target': StandardScaler(axis=(0, 1))}
# scalers = {'target': ScalerModule(scaler=MinMaxScaler())}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
# splitter = TemporalSplitter(val_len=0.0, test_len=0.0)
# splitter = AtTimeStepSplitter(datetime.datetime(2002, 1, 1), datetime.datetime(2003,1,1), datetime.datetime(2002,12, 31), datetime.datetime(2003,12,31))

train_dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    # scalers=scalers,
    # splitter=splitter,
    batch_size=5,
    # workers=8
    pin_memory=True,
)
# test_dm = SpatioTemporalDataModule(
#     dataset=test_dataset,
#     scalers=scalers,
#     splitter=splitter,
#     batch_size=50,
#     # workers=1
# )
validation_dm = SpatioTemporalDataModule(
    dataset=validation_dataset,
    # scalers=scalers,
    # splitter=splitter,
    batch_size=5,
    # workers=8
    pin_memory=True,
)
train_dm.setup()
validation_dm.setup()



In [71]:
import snntorch as snn
s = snn.Synaptic(
            # alpha=torch.Tensor(in_size),
            # beta=torch.Tensor(in_size),
            alpha=0.9,
            beta=0.8,
            learn_alpha=True,
            learn_beta=True,
            learn_threshold=True,
            init_hidden=True,
            output=True
            )

In [66]:
t1, t2 = torch.Tensor(2,3), torch.Tensor(2,3)

In [256]:
s(t2)

(tensor([[0., 0., 0.],
         [0., 0., 0.]], grad_fn=<MulBackward0>),
 tensor([[-6.4656e+34,  4.5616e-40, -6.4656e+34],
         [ 4.5616e-40,  4.4141e-43,  0.0000e+00]], grad_fn=<SubBackward0>),
 tensor([[-3.2336e+34,  2.2814e-40, -3.2336e+34],
         [ 2.2814e-40,  2.2141e-43,  0.0000e+00]], grad_fn=<SubBackward0>))

In [258]:
q1,w1,e1 = s(t1)
q2,w2,e2 = s(t2)

In [259]:
w1, w2

(tensor([[-5.2954e+34,  3.7361e-40, -5.2954e+34],
         [ 3.7361e-40,  4.4141e-43,  0.0000e+00]], grad_fn=<SubBackward0>),
 tensor([[-5.4126e+34,  3.8187e-40, -5.4126e+34],
         [ 3.8187e-40,  4.4141e-43,  0.0000e+00]], grad_fn=<SubBackward0>))

NameError: name 'dl' is not defined

In [270]:
b1

StaticBatch(
  input=(x=[b=5, t=112, n=2048, f=65], edge_index=[2, e=19328], edge_weight=[e=19328]),
  target=(y=[b=5, t=40, n=2048, f=65]),
  has_mask=False
)

In [34]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [35]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [64]:
1e-5

1e-05

In [19]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)
        
        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

In [20]:
from typing import Optional

from torch import Tensor

from tsl.nn.layers.graph_convs.dense_graph_conv import DenseGraphConvOrderK
from tsl.nn.layers.recurrent import DenseDCRNNCell

from tsl.nn.blocks.encoders.recurrent.base import RNNBase


class DenseDCRNN(RNNBase):
    """Dense implementation of the Diffusion Convolutional Recurrent Neural
    Network from the paper `"Diffusion Convolutional Recurrent Neural Network:
    Data-Driven Traffic Forecasting" <https://arxiv.org/abs/1707.01926>`_
    (Li et al., ICLR 2018).

    In this implementation, the adjacency matrix is dense and the convolution is
    performed with matrix multiplication.

    Args:
        input_size: Size of the input.
        hidden_size: Number of units in the hidden state.
        n_layers: Number of layers.
        k: Size of the diffusion kernel.
        root_weight: Whether to learn a separate transformation for the central
            node.
    """
    _n_states = 1

    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 n_layers: int = 1,
                 cat_states_layers: bool = False,
                 return_only_last_state: bool = False,
                 k: int = 2,
                 root_weight: bool = False,
                 device=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.k = k
        self.device = device
        rnn_cells = [
            DenseDCRNNCell(input_size if i == 0 else hidden_size,
                           hidden_size,
                           k=k,
                           root_weight=root_weight) for i in range(n_layers)
        ]
        super(DenseDCRNN, self).__init__(rnn_cells, cat_states_layers,
                                         return_only_last_state)

    def forward(self, x: Tensor, adj, h: Optional[Tensor] = None, **kwargs):
        """"""
        support = DenseGraphConvOrderK.compute_support(adj, device=self.device)
        return super(DenseDCRNN, self).forward(x, h=h, support=support)


In [21]:
import torch.nn as nn

# from tsl.nn.blocks.encoders import DenseDCRNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class DenseDCRNNThenDiffConvModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 temporal_layers: int = 1,
                 gnn_kernel: int = 2,
                 adjacency = None,
                 device=None):
        super(DenseDCRNNThenDiffConvModel, self).__init__()
        self.adjacency = torch.Tensor(adjacency)
        self.device = device
        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        # self.time_nn = RNN(input_size=hidden_size,
        #                    hidden_size=hidden_size,
        #                    n_layers=rnn_layers,
        #                    cell='gru',
        #                    return_only_last_state=True)
        self.time_nn = DenseDCRNN(input_size=hidden_size,
                                  hidden_size=hidden_size,
                                  n_layers=temporal_layers,
                                  cat_states_layers=True,
                                  return_only_last_state=True,
                                  device=device)
        
        self.space_nn = DiffConv(in_channels=hidden_size * temporal_layers,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb, self.adjacency)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

In [22]:
from tsl.nn.layers.base import TemporalSelfAttention

class TemporalSelfAttentionDiffConv(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 attention_embed_dim: int = 32,
                 n_attn_heads: int = 8,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        # self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = TemporalSelfAttention(attention_embed_dim, n_attn_heads, -1)
        
        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

In [23]:
input_size = torch_dataset.n_channels   # n channel
n_nodes = torch_dataset.n_nodes         # n nodes
horizon = torch_dataset.horizon         # n prediction time steps

In [24]:
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param


stgnn = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
stgnn

TimeThenSpaceModel(
  (encoder): Linear(in_features=65, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=2048, embedding_size=32)
  (time_nn): RNN(
    (rnn): GRU(32, 32)
  )
  (space_nn): DiffConv(32, 32)
  (decoder): Linear(in_features=32, out_features=2600, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=40)
)

In [25]:
model = stgnn

In [26]:
from tsl.nn.models import TransformerModel

model = TransformerModel(input_size=input_size, output_size=input_size, horizon=horizon, hidden_size=64, ff_size=64, n_heads=2, n_layers=1, axis='both')

selected axis both
in both branch


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DenseDCRNNThenDiffConvModel(
    input_size,
    n_nodes=n_nodes,
    horizon=horizon,
    temporal_layers=5,
    adjacency=train_loader.adjacency,
    device=device
    )

In [2]:
from lightning.pytorch.loggers import TensorBoardLogger

In [28]:
from tsl.nn.models import GraphWaveNetModel
model = GraphWaveNetModel(
    input_size=input_size,
    output_size=input_size,
    horizon=horizon,
    n_nodes=n_nodes)

In [29]:
model

GraphWaveNetModel(
  (source_embeddings): NodeEmbedding(n_nodes=2048, embedding_size=10)
  (target_embeddings): NodeEmbedding(n_nodes=2048, embedding_size=10)
  (input_encoder): Linear(in_features=65, out_features=32, bias=True)
  (tconvs): ModuleList(
    (0): TemporalConvNet(
      (convs): ModuleList(
        (0): GatedTemporalConv(
          (pad_layer): ZeroPad2d((0, 0, 0, 0))
          (conv): Conv2d(32, 64, kernel_size=(1, 2), stride=(1, 1))
        )
      )
      (f): Identity()
      (dropout): Identity()
    )
    (1): TemporalConvNet(
      (convs): ModuleList(
        (0): GatedTemporalConv(
          (pad_layer): ZeroPad2d((0, 0, 0, 0))
          (conv): Conv2d(32, 64, kernel_size=(1, 2), stride=(1, 1), dilation=(1, 2))
        )
      )
      (f): Identity()
      (dropout): Identity()
    )
    (2): TemporalConvNet(
      (convs): ModuleList(
        (0): GatedTemporalConv(
          (pad_layer): ZeroPad2d((0, 0, 0, 0))
          (conv): Conv2d(32, 64, kernel_size=(1, 2

In [1]:
from load_data import get_data
from DataLoader import WeatherDL
import pickle
min_max = pickle.load(open('./min_max_test.pkl', 'rb'))
rd = get_data()
# dl = WeatherDL(rd, '2001', batch_size=5, min=min_max['min'],
#         max=min_max['max'])
# next(iter(dl.data_loader))

In [2]:
dl = WeatherDL(rd, '2001', batch_size=1, min=min_max['min'], max=min_max['max'], shuffle=True)

weather loader init
WeatherDataLoader done
weather loader exists
SpatioTemporalDataset init
SpatioTemporalDataset exists
StaticGraphLoader init
StaticGraphLoader exists


In [3]:
iterable = iter(dl.data_loader)
b1 = next(iterable)
b2 = next(iterable)

In [4]:
b1.input

StorageView(x=[1, 112, 2048, 65], edge_index=[2, 19328], edge_weight=[19328])

In [5]:
dl.spatio_temporal_dataset.edge_weight

tensor([1.0000e+00, 3.1318e-05, 3.1318e-05,  ..., 1.5112e-05, 9.7185e-06,
        7.0285e-06])

In [6]:
from models.TeporalSynapticLeanableWeights import TemporalSynapticLearnableWeights
input_size = dl.spatio_temporal_dataset.n_channels   # n channel
n_nodes = dl.spatio_temporal_dataset.n_nodes         # n nodes
horizon = dl.spatio_temporal_dataset.horizon  
output_type = "membrane_potential"   
model = TemporalSynapticLearnableWeights(
    input_size=input_size,
    n_nodes=n_nodes,
    horizon=horizon,
    # horizon=8,5
    hidden_size=128,
    output_type=output_type,
    number_of_blocks=1,
    number_of_temporal_steps=3,
    edge_index=dl.spatio_temporal_dataset.edge_index.to('cuda:0'),
    edge_weights=dl.spatio_temporal_dataset.edge_weight.to('cuda:0')
)


# from tensorboardX import SummaryWriter

# writer = SummaryWriter('summary/Presentation2')
# writer.add_graph(model, b1.input.x)
# # writer.add_graph(model, [[b1.input.x, b1.input.edge_index, b1.input.edge_weight]])

In [14]:
!pip install torchview

In [1]:
# from torchview import draw_graph

# model_graph = draw_graph(model, input_size=(1, 112, n_nodes, 65))

In [8]:
model = model.to('cuda:0')
b1.input.x=b1.input.x.to('cuda:0')
y = model(b1.input.x)

In [9]:
from torchviz import make_dot

make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True).render('graph_viz', format='png')

KeyboardInterrupt: 

In [19]:
from torchview import draw_graph
model_graph = draw_graph(model, input_size=b1.input.x.size(), expand_nested=True)
model_graph.visual_graph

RuntimeError: Failed to run torchgraph see error message

In [27]:
(b1.target.y == b2.target.y).all()

tensor(False)

In [72]:
b1.input.x.size()

torch.Size([5, 112, 2048, 65])

In [100]:
weight = torch.Tensor(112, 2048, 15)

In [114]:
repeats = [-1] * (len(weight.shape))
repeats 

[-1, -1, -1]

In [115]:
weight.expand(*repeats).size()

torch.Size([112, 2048, 15])

In [108]:
torch.cat((b1.input.x, weight.expand(*repeats)), dim=-1)

RuntimeError: Sizes of tensors must match except in dimension 3. Expected size 5 but got size 0 for tensor number 1 in the list.

In [93]:
torch.cat([b1.input.x, weight], dim=-1).size()

RuntimeError: Tensors must have same number of dimensions: got 4 and 3

In [155]:
a = torch.randn(5, 112, 2048, 65)
b = torch.randn(1, 112, 2048, 15)

repeat_vals = [a.shape[0] // b.shape[0]] + [-1] * (len(b.shape) - 1)
# or directly repeat_vals = (15, -1, -1) or (15, 200, 2048) if shapes are known and fixed...
res = torch.cat((a, b.expand(*repeat_vals)), dim=-1)
print(res.shape)
repeat_vals

torch.Size([5, 112, 2048, 80])


[5, -1, -1, -1]

In [144]:
a = torch.Tensor(2048,65)
# a.expand(*b1.input.x.size()).size()
torch.nn.Parameter(a).unsqueeze(0).unsqueeze(0).size()

torch.Size([1, 1, 2048, 65])

In [133]:
b1.input.x.dim()

4

In [80]:
b1.input.x.size()

torch.Size([5, 112, 2048, 65])

In [183]:
class LearnableWeights(torch.nn.Module):
    def __init__(self, n_nodes: int, learnable_weight_dim: int) -> None:
        super(LearnableWeights, self).__init__()
        self.weights = torch.nn.Parameter(torch.Tensor(n_nodes, learnable_weight_dim))
        
    def forward(self, x):
        # batch, time, nodes, learnable_weights
        b,t, _, _  = x.size()
        return torch.cat([x, self.weights.expand(b,t, -1, -1)], dim=-1)

In [184]:
learnable = LearnableWeights(2048, 20)
learnable(b1.input.x).size()

torch.Size([5, 112, 2048, 85])

In [258]:
import torch
from torch.nn import Module

class MultiParam(Module):
    def __init__(self, model, args_to_take, return_original_x = False):
        self.model = model
        self.args_to_take = args_to_take
        self.return_original_x = return_original_x
        
        
    def forward(self, args):
        original_x = args[0]
        print('args', args, self.args_to_take)
        fn_args = [args[k] for k in self.args_to_take]
        
        res = self.model(*fn_args)
        args[0] = res
        if self.return_original_x:
            return [*args, original_x]
        return args
    
    _backward_hooks = False
    _backward_pre_hooks = False
    _forward_hooks = _forward_pre_hooks = False
    

In [263]:
a= [1,2,3,4]
[a[k] for k in [-1, 1]]

[4, 2]

In [259]:
def fn(a, b, c):
    return a + b + c

def fn2(a, b):
    return a + b

In [260]:
t = torch.nn.Sequential(MultiParam(fn, [0, 2,3]), MultiParam(fn2, [0,1]))

In [261]:
t([1,2,3,4])

args [1, 2, 3, 4] [0, 2, 3]
args [8, 2, 3, 4] [0, 1]


[10, 2, 3, 4]

In [68]:
torch.tensor((320**2) * 0.03, dtype=torch.int16)

tensor(3072, dtype=torch.int16)

In [6]:
from tsl.metrics.torch.metric_base import MaskedMetric
from torch.nn import functional as F
import torch

def rmse_loss(*args, **kwargs):
    print(args[0].size(), args[1].size())
    return torch.sqrt(F.mse_loss(*args, **kwargs) + 1e-9)
    
class MaskedRMSE(MaskedMetric):
    """Root Mean Squared Error Metric.

    Args:
        mask_nans (bool, optional): Whether to automatically mask nan values.
        mask_inf (bool, optional): Whether to automatically mask infinite
            values.
        at (int, optional): Whether to compute the metric only w.r.t. a certain
            time step.
    """

    is_differentiable: bool = True
    higher_is_better: bool = False
    full_state_update: bool = False
    

    def __init__(self,
                 mask_nans=False,
                 mask_inf=False,
                 at=None,
                 **kwargs):
        super(MaskedRMSE, self).__init__(metric_fn=rmse_loss,
                                        mask_nans=mask_nans,
                                        mask_inf=mask_inf,
                                        metric_fn_kwargs={'reduction': 'none'},
                                        at=at,
                                        **kwargs)

In [7]:
from collections import Counter

def multi_dot(*arrays, dims = None):

    # arrays = [named_z500, weights]
    all_dims = []
    for tens in arrays:
        all_dims += [d for d in tens.names if d not in all_dims]
    all_dims

    einsum_axes = "abcdefghijklmnopqrstuvwxyz"
    dim_map = {d: einsum_axes[i] for i, d in enumerate(all_dims)}
    dim_map
    common_dims = set.intersection(*(set(arr.names) for arr in arrays))
    common_dims

    # dims = ['lat', 'lon']
    if dims is ...:
            dims = all_dims
    elif isinstance(dims, str):
        dims = (dims,)
    elif dims is None:
        # find dimensions that occur more than one times
        dim_counts: Counter = Counter()
        for arr in arrays:
            dim_counts.update(arr.dims)
        dims = tuple(d for d, c in dim_counts.items() if c > 1)

    dot_dims = set(dims)
    broadcast_dims = common_dims - dot_dims
    broadcast_dims

    input_core_dims = [
        [d for d in arr.names if d not in broadcast_dims] for arr in arrays
    ]
    output_core_dims = [
        [d for d in all_dims if d not in dot_dims and d not in broadcast_dims]
    ]
    subscripts_list = [
        "..." + "".join(dim_map[d] for d in ds) for ds in input_core_dims
    ]
    subscripts = ",".join(subscripts_list)
    subscripts += "->..." + "".join(dim_map[d] for d in output_core_dims[0])

    return torch.einsum(subscripts, *[a.rename(None) for a in arrays])

def weighted_sum(data, weights, dims=None):
    return multi_dot(data, weights, dims=dims)

def sum_of_weights(data, weights, dims=None):
    summed_weights = multi_dot((~torch.isnan(data)).int().float(), weights, dims=dims)
    valid_weights = summed_weights != 0.0
    
    return summed_weights.where(valid_weights, 1e-15)

def weighted_mean(data, weights, dims=None):
    weighted_sum_ = weighted_sum(data, weights, dims=dims)
    sum_o_w = sum_of_weights(data, weights, dims=dims)

    return weighted_sum_ / sum_o_w

def _spatial_average_l2_norm(diff, weights):
    return torch.sqrt(weighted_mean(diff**2, weights, dims=['lat', 'lon']))

In [8]:
variable_set_idx = {v: k+1 for k,v in dict(enumerate(['z', 'q', 't', 'u', 'v'])).items()}
levels = [50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
            1000]
level_idx = {v: k for k,v in dict(enumerate(levels)).items()}

In [4]:
from dataclasses import dataclass

@dataclass
class WeatherVariable:
    variable: str
    level: int


def select_variable(definition: WeatherVariable):
    level_skips = variable_set_idx[definition.variable]
    level_index = level_idx[definition.level]
    
    return ((len(levels) - 1) * level_skips) + level_index
    

In [5]:
import torch
from einops.layers.torch import Rearrange
from metrics.metric_utils import WeatherVariable, select_variable, _spatial_average_l2_norm
from tsl.metrics.torch.metric_base import MaskedMetric


class _WeightedRMSE(torch.nn.Module):
    def __init__(self, weights, variables: list[WeatherVariable], at=None, lat_size=32, lon_size=64) -> None:
        super(_WeightedRMSE, self).__init__()
        self.weights = weights
        self.at = at
        self.variables = [select_variable(variable) for variable in variables]
        self.restore_shape = Rearrange('b t (lat lon) f -> b t lat lon f', lat=lat_size, lon=lon_size)
        
    # def forward(self, predictions:Any, ground_truth: Any):
    def forward(self, *args, **kw_args):
        predictions = args[0]
        ground_truth = args[1]
        
        if self.at != None:
            predictions, ground_truth = predictions[:, self.at], ground_truth[:, self.at]
            
        self.weights.to(predictions.device)
        diff = (predictions[..., self.variables] - ground_truth[...,self.variables])
        diff = self.restore_shape(diff).refine_names('batch', 'time', 'lat', 'lon', 'variables')
        return _spatial_average_l2_norm(diff, self.weights.refine_names('lat')).mean()
    
    def __call__(self, *args, **kw_args):
        return self.forward(*args, **kw_args)
        
class WeightedRMSE(MaskedMetric):
    def __init__(self,
                weights,
                variables: list[WeatherVariable],
                at=None,
                lat_size=32,
                lon_size=64,
                mask_nans=True,
                mask_inf=True,
                **kwargs
                 ):
        
        is_differentiable: bool = True
        higher_is_better: bool = False
        full_state_update: bool = False
        super(WeightedRMSE, self).__init__(metric_fn=_WeightedRMSE(
                                            weights=weights,
                                            variables=variables,
                                            lat_size=lat_size,
                                            lon_size=lon_size,
                                            ),
                                        mask_nans=mask_nans,
                                        mask_inf=mask_inf,
                                        metric_fn_kwargs={'reduction': 'none'},
                                        at=at,
                                        **kwargs)
        

In [6]:
import inspect
inspect.getfullargspec(WeightedRMSE).args

['self',
 'weights',
 'variables',
 'at',
 'lat_size',
 'lon_size',
 'mask_nans',
 'mask_inf']

In [7]:
dl.spatio_temporal_dataset.n_nodes, dl.spatio_temporal_dataset.edge_index.size(), dl.spatio_temporal_dataset.edge_weight.size()

(2048, torch.Size([2, 19328]), torch.Size([19328]))

In [8]:
(2048**2) // (19328 * 100)

2

In [9]:
19328 // 2048

9

In [10]:
wrmse = WeightedRMSE(
    dl.data_wrapper.node_weights,
    variables=[
        # WeatherVariable('z', 500),
        WeatherVariable('t', 850)
        ]
    )

In [11]:
from metrics.weighted_rmse import WeightedRMSE
from metrics.metric_utils import WeatherVariable

In [33]:
rmse = WeightedRMSE(
    dl.data_wrapper.node_weights,
    variables=[
        WeatherVariable('z', 500),
        WeatherVariable('t', 850)
        ]
    )
rmse(b1.target.y[:, :1], b1.target.y[:, 39:40])

tensor(0.1509)

In [18]:
isinstance(wrmse, MaskedMetric)

True

In [17]:
wrmse(b1.target.y, b2.target.y)

tensor(0.0714)

In [43]:
b1_vars = b1.target.y[..., wrmse.variables]
b2_vars = b2.target.y[..., wrmse.variables]

In [50]:
b1

StaticBatch(
  input=(x=[b=5, t=112, n=2048, f=65], edge_index=[2, e=19328], edge_weight=[e=19328]),
  target=(y=[b=5, t=40, n=2048, f=65]),
  has_mask=False
)

In [51]:
(b1.input.x == b2.input.x).all()

tensor(True)

In [17]:
b1_vars

NameError: name 'b1_vars' is not defined

In [47]:
b2_vars

tensor([[[[ 1.1067e-04,  4.7427e+00],
          [ 1.1612e-04,  4.5044e+00],
          [ 1.1711e-04,  4.4789e+00],
          ...,
          [ 2.7888e-05, -1.4735e+01],
          [ 2.9869e-05, -1.3663e+01],
          [ 3.2349e-05, -1.2503e+01]],

         [[ 9.4805e-05,  3.4746e+00],
          [ 9.8275e-05,  3.4236e+00],
          [ 1.0472e-04,  3.5966e+00],
          ...,
          [ 2.8383e-05, -1.0781e+01],
          [ 2.9374e-05, -9.7567e+00],
          [ 3.1358e-05, -8.7099e+00]],

         [[ 8.6378e-05,  2.4363e+00],
          [ 8.8362e-05,  2.5214e+00],
          [ 9.0839e-05,  2.8363e+00],
          ...,
          [ 3.1358e-05, -7.7510e+00],
          [ 3.2844e-05, -6.9198e+00],
          [ 3.4332e-05, -6.0602e+00]],

         ...,

         [[ 7.7952e-05, -5.6545e+00],
          [ 7.5474e-05, -4.9878e+00],
          [ 7.5474e-05, -4.0744e+00],
          ...,
          [ 4.5236e-05, -3.4730e+00],
          [ 3.9784e-05, -1.8418e+00],
          [ 3.3340e-05, -3.1266e-01]],

     

In [25]:
loss = MaskedRMSE()
loss(b1.target.y, b2.target.y)

torch.Size([5, 40, 2048, 65]) torch.Size([5, 40, 2048, 65])


tensor(3.1623e-05)

In [19]:
F.mse_loss(b1.target.y, b2.target.y)

tensor(0.)

In [31]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE, MaskedMSE
from tsl.engines import Predictor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

loss_fn = MaskedMSE()
steps_per_day = 24 // 3
metrics = {'mse': MaskedMSE(),
        #    'mape': MaskedMAPE(),
           'mse_at_3_days': MaskedMSE(at=(steps_per_day * 3) - 1),  # 'steps_per_day * 3' indicates the 24th time step,
                                          # which correspond to 3 days ahead
           'mse_at_5_days': MaskedMSE(at=(steps_per_day * 5) - 1)}

# loss_fn = MaskedRMSE()
# steps_per_day = 24 // 3
# metrics = {'rmse': MaskedRMSE(),
#         #    'mape': MaskedMAPE(),
#            'rmse_at_3_days': MaskedRMSE(at=(steps_per_day * 3) - 1),  # 'steps_per_day * 3' indicates the 24th time step,
#                                           # which correspond to 3 days ahead
#            'rmse_at_5_days': MaskedRMSE(at=(steps_per_day * 5) - 1)}

# setup predictor
# stgnn = stgnn.to(torch.device(device))
predictor = Predictor(
    model=model,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)
device, torch.cuda.device_count(), model._get_name()

(device(type='cuda', index=0), 1, 'GraphWaveNetModel')

In [32]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir="logs", name=f"{model._get_name()}_mse_epochs", version=0,)

# import pytorch_lightning as pl
import lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mse',
    mode='min',
)

trainer = pl.Trainer(max_epochs=2,
                     logger=logger,
                    #  gpus=1 if torch.cuda.is_available() else None,
                    devices=1,
                    accelerator='gpu',
                    #  limit_train_batches=30,
                     limit_val_batches=50,
                     callbacks=[checkpoint_callback], num_sanity_val_steps=0)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
# import os
# os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'

In [34]:
# torch.set_float32_matmul_precision('medium' | 'high')
torch.set_float32_matmul_precision('medium')
trainer.fit(predictor, train_dataloaders=stg_train, val_dataloaders=stg_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | loss_fn       | MaskedMSE         | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | test_metrics  | MetricCollection  | 0     
4 | model         | GraphWaveNetModel | 1.7 M 
----------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.671     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/1724 [00:00<?, ?it/s] 

Only args ['edge_index', 'edge_weight', 'x'] are forwarded to the model (GraphWaveNetModel). 


Epoch 0:   9%|▉         | 160/1724 [05:16<51:35,  0.51it/s, v_num=0] 

In [32]:
5*2048, 5*160

(10240, 800)

In [23]:
import gc
model = None
gc.collect()
# with torch.no_grad():
torch.cuda.empty_cache()

In [20]:
trainer.fit(predictor, train_dataloaders=train_dm.get_dataloader(), val_dataloaders=validation_dm.get_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | loss_fn       | MaskedMSE         | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | test_metrics  | MetricCollection  | 0     
4 | model         | GraphWaveNetModel | 1.7 M 
----------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.671     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/1724 [00:00<?, ?it/s] 

AssertionError: Caught AssertionError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/arturas/anaconda3/envs/weather/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/arturas/anaconda3/envs/weather/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 53, in fetch
    data = self.dataset[possibly_batched_index]
           ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4046896/1010229011.py", line 376, in __getitem__
    item = self.get(item)
           ^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4046896/1010229011.py", line 313, in get
    self._add_to_sample(sample, WINDOW, 'input', time_index=wdw_idxs)
  File "/tmp/ipykernel_4046896/1010229011.py", line 819, in _add_to_sample
    tensor, scaler = self.get_tensor(item.keys[0],
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_40468

In [19]:
trainer.fit(predictor, train_dm)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | loss_fn       | MaskedMSE         | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | test_metrics  | MetricCollection  | 0     
4 | model         | GraphWaveNetModel | 1.7 M 
----------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.671     Total estimated model params size (MB)


TypeError: An invalid dataloader was returned from `SpatioTemporalDataModule.train_dataloader()`. Found None.

In [27]:
batch = next(iter(train_dm.get_dataloader()))

In [28]:
batch.input

StorageView(x=[5, 112, 2048, 65], edge_index=[2, 19328], edge_weight=[19328])

In [30]:

dcrnn = DenseDCRNN(input_size=hidden_size,
                            hidden_size=hidden_size,
                            n_layers=2,
                            cat_states_layers=True,
                            return_only_last_state=True,
                            device='cpu')
# x = batch.input.x.to('cpu')
# adj = torch.Tensor(train_loader.adjacency_weighted).to('cpu')
# support = DenseGraphConvOrderK.compute_support(adj, device='cpu')
encoder = nn.Linear(input_size, hidden_size)

# node_embeddings = NodeEmbedding(n_nodes, hidden_size)
x = encoder(batch.input.x)
res = dcrnn(x, torch.Tensor(train_loader.adjacency_weighted))

In [55]:
space_nn = DiffConv(in_channels=hidden_size * 2,
                                 out_channels=hidden_size,
                                 k=5)
space_nn, hidden_size

(DiffConv(96, 32), 32)

In [57]:
# 1 temp layer size - torch.Size([5, 2048, 32])
# 1 temp layer size - torch.Size([5, 2048, 64])
res.size()

torch.Size([5, 2048, 64])

In [58]:
space_res = space_nn(res, train_loader.edge_index, train_loader.edge_weight)
space_res.size()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10240x704 and 1056x32)

In [54]:
decoder = nn.Linear(hidden_size, input_size * horizon)
decorer_res = decoder(space_res)
decorer_res.size()

torch.Size([5, 2048, 2600])

In [56]:
rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)
horizont = rearrange(decorer_res)
horizont.size()

torch.Size([5, 40, 2048, 65])

In [57]:
model = DenseDCRNNThenDiffConvModel(
    input_size,
    n_nodes=n_nodes,
    horizon=horizon,
    temporal_layers=1,
    adjacency=train_loader.adjacency,
    device='cpu'
    )

In [58]:
model_res = model(*batch.input)

h: torch.Size([5, 2048, 32])


In [36]:
attn = TemporalSelfAttention(64, 8, -1)
res = attn(batch.input.x)

In [37]:
res[0].shape, res[1].shape

(torch.Size([10, 112, 2048, 64]), torch.Size([10, 2048, 112, 112]))